# Availability Aware Routing

In [6]:
import networkx as nx
import plotly.graph_objects as go
import plotly.colors
import plotly.io as pio
import plotly.express as px
import pandas as pd
import statistics
import random
import os
import time
from copy import deepcopy
from dash import Dash, html, dcc, Input, Output, callback
import pickle
from pprint import pprint
from my_utils import *

## Measures

### Without trimming

#### Initial Solutions

Set test parameters

In [7]:
graph_names = ["22_optic_eu_latlon", "25_interoute_latlon", "26_usanet_latlon", "28_optic_eu_latlon", "37_eu_latlon", "39_optic_north_american_latlon", "79_optic_nfsnet_latlon"]
methods = ["min_cutting_prob", "min_prob"]
k = 10

Test paramters are from previous measures (INFOCOM)

In [ ]:
# Load the results
initial_solutions = {}
path = os.path.join("results", "initial_solutions.pkl")
with open(path, "rb") as f:
    initial_solutions = pickle.load(f)

# graph_names = ["22_optic_eu_latlon", "26_usanet_latlon", "28_optic_eu_latlon", "37_eu_latlon", "39_optic_north_american_latlon", "79_optic_nfsnet_latlon"]
# methods = ["min_cutting_prob", "min_prob"]
# k = 10

# infocom_test = {}

# for graph_name in graph_names:
#     # Read the graph and SRLGs
#     input_graph = os.path.join("..", "korábbi repók", "JSACtoGitHub", "Networks", f"{graph_name}.gml")
#     srlg_path = os.path.join("..", "korábbi repók", "JSACtoGitHub", "PSRLG lists", f"{graph_name}_simple_it6.xml")
#     G = read_graph_from_gml(input_graph)
#     G.graph["srlgs"] = read_srlgs_from_xml(srlg_path, G)
#     infocom_test[graph_name] = {"graph": G, "terminals": {}}

#     # Loop through distances
#     nodes = list(G.nodes)
#     terminals = random.sample([(u, v) for u in nodes for v in nodes if u != v and u < v], 50)
#     for s, t in terminals:
#         infocom_test[graph_name]["terminals"][(s, t)] = {}


run for initial solutions

In [26]:
# runtime: 14 min
exe_path = os.path.join("cpp_codes", "dateline_prob.exe")

# For all graphs
for graph_name in graph_names:
    print(f"Processing graph: {graph_name}")
    G = initial_solutions[graph_name]["graph"]

    # For all terminal pairs
    for i, (s, t) in enumerate(initial_solutions[graph_name]["terminals"].keys()):
        # For all methods
        # print(f"\tFinding paths for source: {s} ({G.nodes[s]['label']}), target:, {t} ({G.nodes[t]['label']})")
        if i % 10 == 0:
            print(i, end=', ')
        for method in methods:
            
            # Initialize the solution dictionary for this method
            solution = initial_solutions[graph_name]['terminals'][(s,t)][method] = {}
            solution['surviving_probabilities'] = []
            solution['walks'] = []
            solution['srlgs'] = []

            # Set all capacities to 1
            for srlg in G.graph['srlgs']:
                srlg['capacity'] = 1

            # C++ code needs integer coordinates, so scale positions
            G_scaled = G.copy()
            for node, data in G_scaled.nodes(data=True):
                G_scaled.nodes()[node]['pos'] = (round(data['pos'][0]*100), round(data['pos'][1]*100))

            # Solve
            input_string = generate_input_string_with_prob(G_scaled, s, t)
            number_of_backups = 0
            if method[-1].isdigit():
                number_of_backups = int(method[-1])
                method = method[:-2]
            output = solve_from_string_prob(exe_path, input_string, method = method, number_of_backups=number_of_backups)
            for sol_dict in output.values():
                solution['walks'].append(sol_dict['paths'])
                solution['surviving_probabilities'].append(get_surviving_probabilities_of_walks(G, solution["walks"][-1]))
                solution['srlgs'].append(deepcopy(G.graph["srlgs"]))
                for srlg_id, srlg_capacity in sol_dict['srlgs'].items():
                    solution['srlgs'][-1][srlg_id]['capacity'] = srlg_capacity

    print()

# Save the results
path = os.path.join("results", "initial_solutions.pkl")
with open(path, "wb") as f:
    pickle.dump(initial_solutions, f)

Processing graph: 22_optic_eu_latlon
0, 10, 20, 30, 40, 
Processing graph: 25_interoute_latlon
0, 10, 20, 30, 40, 
Processing graph: 26_usanet_latlon
0, 10, 20, 30, 40, 
Processing graph: 28_optic_eu_latlon
0, 10, 20, 30, 40, 
Processing graph: 37_eu_latlon
0, 10, 20, 30, 40, 
Processing graph: 39_optic_north_american_latlon
0, 10, 20, 30, 40, 
Processing graph: 79_optic_nfsnet_latlon
0, 10, 20, 30, 40, 


#### Improve Solutions

In [27]:
#runtime: 8 min

# Load the results
initial_solutions = {}
path = os.path.join("results", "initial_solutions.pkl")
with open(path, "rb") as f:
    initial_solutions = pickle.load(f)

improved_solutions = {
    graph_name: {
        "graph": initial_solutions[graph_name]["graph"],
        "terminals": {
            (s, t): {
                method: improve_all_surviving_probabilities(
                    initial_solutions[graph_name]["graph"],
                    initial_solutions[graph_name]["terminals"][(s, t)][method]
                )
                for method in initial_solutions[graph_name]["terminals"][(s, t)].keys()
            }
            for (s, t) in initial_solutions[graph_name]["terminals"].keys()
        }
    }
    for graph_name in initial_solutions.keys()
}

# Save the improved solutions
path = os.path.join("results", "improved_solutions.pkl")
with open(path, "wb") as f:
    pickle.dump(improved_solutions, f)

#### Shorten Paths

without modifying capacities

In [28]:
#runtime: 6 min

# Load the results
improved_solutions = {}
path = os.path.join("results", "improved_solutions.pkl")
with open(path, "rb") as f:
    improved_solutions = pickle.load(f)

short = {
    graph_name: {
        "graph": improved_solutions[graph_name]["graph"],
        "terminals": {
            (s, t): {
                method: {} for method in improved_solutions[graph_name]["terminals"][(s, t)].keys()
            } for (s, t) in improved_solutions[graph_name]["terminals"].keys()
        }
    } for graph_name in improved_solutions.keys()
}

for graph_name in improved_solutions.keys():
    print(f"Shortening paths for graph: {graph_name}")
    G = improved_solutions[graph_name]["graph"]
    i = 0
    for (s, t) in improved_solutions[graph_name]["terminals"].keys():
        i += 1
        print(i, end=', ')
        for method in improved_solutions[graph_name]["terminals"][(s, t)].keys():
            original_solutions = improved_solutions[graph_name]["terminals"][(s, t)][method]
            shortened_solutions = deepcopy(original_solutions)
            for n in range(1, 11):
                for k in range(1, n+1):
                    
                    og_paths = original_solutions[(n, k)]["walks"]
                    og_srlgs = original_solutions[(n, k)]["srlgs"]
                    short_paths = shorten_paths(G, s, t, og_paths, og_srlgs)

                    og_len = sum(len(walk) - 1 for walk in og_paths)
                    short_len = sum(len(walk) - 1 for walk in short_paths)
                    if short_len < og_len:
                        shortened_solutions[(n, k)]["walks"] = short_paths
                        shortened_solutions[(n, k)]["srlgs"] = og_srlgs
                        shortened_solutions[(n, k)]["surviving_probability"] = get_surviving_probabilities_of_walks(G, short_paths)[k]
            short[graph_name]["terminals"][(s, t)][method] = shortened_solutions
    print()

path = os.path.join("results", "shortened_solutions.pkl")
with open(path, "wb") as f:
    pickle.dump(short, f)


Shortening paths for graph: 22_optic_eu_latlon
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 
Shortening paths for graph: 26_usanet_latlon
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 
Shortening paths for graph: 28_optic_eu_latlon
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 
Shortening paths for graph: 37_eu_latlon
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 
Shortening paths for graph: 39_optic_north_american_

#### `min_cutting_prob`

##### Helper Functions

In [3]:
def min_prob_rule(srlgs, number_of_backups=0, max_number_of_paths=10):
    new_srlgs = deepcopy(srlgs)
    eligible_srlgs = [srlg for srlg in new_srlgs if srlg['capacity'] < max_number_of_paths]
    # RULE: Find the SRLG with the minimum probability * capacity
    if not eligible_srlgs:
            return False, new_srlgs
    min_probability_srlg = min(eligible_srlgs, key=lambda srlg: srlg['probability'] * srlg['capacity'])
    min_probability_srlg["capacity"] += 1
    if number_of_backups > 0 and min_probability_srlg["capacity"] > number_of_backups:
        min_probability_srlg["capacity"] = max_number_of_paths
    return True, new_srlgs

def min_prob_batch_rule(srlgs, batch_ratio = 0.05, number_of_backups=0, max_number_of_paths=10):
    # RULE: Find the SRLG with the minimum probability * capacity
    new_srlgs = deepcopy(srlgs)
    for _ in range(max(2, int(len(new_srlgs) * batch_ratio))):
        eligible_srlgs = [srlg for srlg in new_srlgs if srlg['capacity'] < max_number_of_paths]
        if not eligible_srlgs:
            return False, new_srlgs
        min_probability_srlg = min(eligible_srlgs, key=lambda srlg: srlg['probability'] * srlg['capacity'])
        min_probability_srlg["capacity"] += 1
        if number_of_backups > 0 and min_probability_srlg["capacity"] > number_of_backups:
            min_probability_srlg["capacity"] = max_number_of_paths
    return True, new_srlgs


##### Tolerance: 1%, backups: 1

- 22_optice_eu_latlon: 7002 s = 1.9h
- 26_usanet_latlon: 7704 s = 2.1h
- 28_optic_eu_latlon: 5374 s = 1.5 h
- 37_eu_latlon: 6540 s = 1.8 h
- 39_optic_nort_american_latlon: 9594 s = 2.6h
- 79_optic_nfsnet_latlon: 25932 s = 7.2 h
- 25_interoute_v2_clean = 8953 s = 2.5 h

In [4]:
exe_path = os.path.join("cpp_codes", "dateline_capacity.exe")
tolerance = 0.01

# Load the results
improved_solutions = {}
path = os.path.join("results", "improved_solutions.pkl")
with open(path, "rb") as f:
    improved_solutions = pickle.load(f)

short_solutions = {
    graph_name: {
        "graph": improved_solutions[graph_name]["graph"],
        "terminals": {
            (s, t): {
                method: {} for method in improved_solutions[graph_name]["terminals"][(s, t)].keys()
            } for (s, t) in improved_solutions[graph_name]["terminals"].keys()
        }
    } for graph_name in improved_solutions.keys()
}

for graph_name in improved_solutions.keys():
    G = improved_solutions[graph_name]["graph"]
    G_scaled = G.copy()
    for node, data in G_scaled.nodes(data=True):
        G_scaled.nodes()[node]['pos'] = (round(data['pos'][0]*100), round(data['pos'][1]*100))
    start_time = time.time()
    print(f"Processing graph: {graph_name}")
    terminal_i = 0
    for s, t in improved_solutions[graph_name]["terminals"].keys():
        terminal_i += 1
        print(f"\tterminal {terminal_i}:")
        # get the corrected solution
        solution = improved_solutions[graph_name]["terminals"][(s, t)]["min_cutting_prob"]
        short_solution = deepcopy(solution)
        # go through each number of paths and backups
        for n in range(2, 11):
            print(f"\t\tnumber of paths: {n}")
            number_of_backups = 1
            k = n - number_of_backups

            # try to shorten the paths
            short_solution[(n, k)]["srlgs"] = make_srlgs_tight(G, solution[(n, k)]["walks"])
            new_srlgs = deepcopy(short_solution[(n, k)]["srlgs"])
            prev_srlgs = deepcopy(new_srlgs)
            short_solution[(n, k)]["walks"] = shorten_paths(G, s, t, solution[(n, k)]["walks"], new_srlgs)
            new_walks = deepcopy(short_solution[(n, k)]["walks"])
            short_solution[(n, k)]["surviving_probability"] = get_surviving_probabilities_of_walks(G, new_walks)[k]
            new_sp = deepcopy(short_solution[(n, k)]["surviving_probability"])
            original_sp = deepcopy(new_sp)

            # Increase SRLG capacities in batches
            batch_i = 0
            are_there_free_srlgs = True
            print(f"\t\t\tbatch iteration: ", end='')
            while (
                len(new_walks) == n and 
                np.log10(1-new_sp) <  np.log10(1-original_sp)*(1-tolerance) and
                are_there_free_srlgs
            ):
                prev_srlgs = deepcopy(new_srlgs)
                batch_i += 1
                print(batch_i, end=', ')
                old_len = sum(len(walk)-1 for walk in short_solution[(n, k)]["walks"])
                new_len = sum(len(walk)-1 for walk in new_walks)

                # If the new solution is better, save it
                if new_len < old_len:
                    short_solution[(n, k)]["srlgs"] = deepcopy(new_srlgs)
                    short_solution[(n, k)]["walks"] = deepcopy(new_walks)
                    short_solution[(n, k)]["surviving_probability"] = deepcopy(new_sp)

                # Try to find a new solution
                are_there_free_srlgs, new_srlgs = min_prob_batch_rule(new_srlgs, number_of_backups=number_of_backups)

                # Generate new input string and solve the problem
                input_string = generate_input_string(G_scaled, s, t, new_srlgs)
                special_path, walks, potentials, cutting_edge_groups = solve_from_string(exe_path, input_string)
                new_walks = shorten_paths(G, s, t, walks, new_srlgs)
                new_sp = get_surviving_probabilities_of_walks(G, new_walks)[k]
            print()

            # Get the last valid solution
            new_srlgs = deepcopy(prev_srlgs)
            input_string = generate_input_string(G_scaled, s, t, new_srlgs)
            special_path, walks, potentials, cutting_edge_groups = solve_from_string(exe_path, input_string)
            new_walks = shorten_paths(G, s, t, walks, new_srlgs)
            new_sp = get_surviving_probabilities_of_walks(G, new_walks)[k]

            # Increase SRLG capacities one by one
            one_by_one_i = 0
            are_there_free_srlgs = True
            print(f"\t\t\tone by one iteration: ", end='')
            while (
                len(new_walks) == n and 
                np.log10(1-new_sp) <  np.log10(1-original_sp)*(1-tolerance) and
                are_there_free_srlgs 
            ):
                one_by_one_i += 1
                print(one_by_one_i, end=', ')
                old_len = sum(len(walk)-1 for walk in short_solution[(n, k)]["walks"])
                new_len = sum(len(walk)-1 for walk in new_walks)

                # If the new solution is better, save it
                if new_len < old_len:
                    short_solution[(n, k)]["srlgs"] = deepcopy(new_srlgs)
                    short_solution[(n, k)]["walks"] = deepcopy(new_walks)
                    short_solution[(n, k)]["surviving_probability"] = deepcopy(new_sp)
                
                # Try to find a new solution
                are_there_free_srlgs, new_srlgs = min_prob_rule(new_srlgs, number_of_backups=0)

                # Generate new input string and solve the problem
                input_string = generate_input_string(G_scaled, s, t, new_srlgs)
                special_path, walks, potentials, cutting_edge_groups = solve_from_string(exe_path, input_string)
                new_walks = shorten_paths(G, s, t, walks, new_srlgs)
                new_sp = get_surviving_probabilities_of_walks(G, new_walks)[k]

            print()
            short_solutions[graph_name]["terminals"][(s, t)]["min_cutting_prob"][(n, k)] = deepcopy(short_solution[(n, k)])
            path = os.path.join("results", f"short_solutions_{tolerance}.pkl")
            with open(path, "wb") as f_out:
                pickle.dump(short_solutions, f_out)
    end_time = time.time()
    print(f"Time taken: {end_time - start_time:.4f} seconds")

Processing graph: 22_optic_eu_latlon
	terminal 1:
		number of paths: 2
			batch iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 
			one by one iteration: 1, 2, 3, 4, 5, 6, 
		number of paths: 3
			batch iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 
			one by one iteration: 1, 2, 
		number of paths: 4
			batch iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 
			one by one iteration: 1, 2, 3, 4, 5, 6, 7, 8, 
		number of paths: 5
			batch iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 
			one by one iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 
		number of paths: 6
			batch iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 
			one by one iteration: 1, 2, 3, 4, 5, 
		number of paths: 7
			batch iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 
			one by one it

##### Tolerance: 1, backups: 2, only for "22_optic_eu_latlon"

runtime: 2.2h

In [46]:
exe_path = os.path.join("cpp_codes", "dateline_capacity.exe")
tolerance = 0.01

# Load the results
short_solutions = {}
path = os.path.join("results", "short_solutions_0.01.pkl")
with open(path, "rb") as f:
    short_solutions = pickle.load(f)

for graph_name in ["22_optic_eu_latlon"]:
    G = improved_solutions[graph_name]["graph"]
    G_scaled = G.copy()
    for node, data in G_scaled.nodes(data=True):
        G_scaled.nodes()[node]['pos'] = (round(data['pos'][0]*100), round(data['pos'][1]*100))
    start_time = time.time()
    print(f"Processing graph: {graph_name}")
    terminal_i = 0
    for s, t in improved_solutions[graph_name]["terminals"].keys():
        terminal_i += 1
        print(f"\tterminal {terminal_i}:")
        # get the corrected solution
        solution = improved_solutions[graph_name]["terminals"][(s, t)]["min_cutting_prob"]
        short_solution = deepcopy(solution)
        # go through each number of paths and backups
        for n in range(3, 11):
            print(f"\t\tnumber of paths: {n}")
            number_of_backups = 2
            k = n - number_of_backups

            # try to shorten the paths
            short_solution[(n, k)]["srlgs"] = make_srlgs_tight(G, solution[(n, k)]["walks"])
            new_srlgs = deepcopy(short_solution[(n, k)]["srlgs"])
            prev_srlgs = deepcopy(new_srlgs)
            short_solution[(n, k)]["walks"] = shorten_paths(G, s, t, solution[(n, k)]["walks"], new_srlgs)
            new_walks = deepcopy(short_solution[(n, k)]["walks"])
            short_solution[(n, k)]["surviving_probability"] = get_surviving_probabilities_of_walks(G, new_walks)[k]
            new_sp = deepcopy(short_solution[(n, k)]["surviving_probability"])
            original_sp = deepcopy(new_sp)

            # Increase SRLG capacities in batches
            batch_i = 0
            are_there_free_srlgs = True
            print(f"\t\t\tbatch iteration: ", end='')
            while (
                len(new_walks) == n and 
                np.log10(1-new_sp) <  np.log10(1-original_sp)*(1-tolerance) and
                are_there_free_srlgs
            ):
                prev_srlgs = deepcopy(new_srlgs)
                batch_i += 1
                print(batch_i, end=', ')
                old_len = sum(len(walk)-1 for walk in short_solution[(n, k)]["walks"])
                new_len = sum(len(walk)-1 for walk in new_walks)

                # If the new solution is better, save it
                if new_len < old_len:
                    short_solution[(n, k)]["srlgs"] = deepcopy(new_srlgs)
                    short_solution[(n, k)]["walks"] = deepcopy(new_walks)
                    short_solution[(n, k)]["surviving_probability"] = deepcopy(new_sp)

                # Try to find a new solution
                are_there_free_srlgs, new_srlgs = min_prob_batch_rule(new_srlgs, number_of_backups=number_of_backups)

                # Generate new input string and solve the problem
                input_string = generate_input_string(G_scaled, s, t, new_srlgs)
                special_path, walks, potentials, cutting_edge_groups = solve_from_string(exe_path, input_string)
                new_walks = shorten_paths(G, s, t, walks, new_srlgs)
                new_sp = get_surviving_probabilities_of_walks(G, new_walks)[k]
            print()

            # Get the last valid solution
            new_srlgs = deepcopy(prev_srlgs)
            input_string = generate_input_string(G_scaled, s, t, new_srlgs)
            special_path, walks, potentials, cutting_edge_groups = solve_from_string(exe_path, input_string)
            new_walks = shorten_paths(G, s, t, walks, new_srlgs)
            new_sp = get_surviving_probabilities_of_walks(G, new_walks)[k]

            # Increase SRLG capacities one by one
            one_by_one_i = 0
            are_there_free_srlgs = True
            print(f"\t\t\tone by one iteration: ", end='')
            while (
                len(new_walks) == n and 
                np.log10(1-new_sp) <  np.log10(1-original_sp)*(1-tolerance) and
                are_there_free_srlgs 
            ):
                one_by_one_i += 1
                print(one_by_one_i, end=', ')
                old_len = sum(len(walk)-1 for walk in short_solution[(n, k)]["walks"])
                new_len = sum(len(walk)-1 for walk in new_walks)

                # If the new solution is better, save it
                if new_len < old_len:
                    short_solution[(n, k)]["srlgs"] = deepcopy(new_srlgs)
                    short_solution[(n, k)]["walks"] = deepcopy(new_walks)
                    short_solution[(n, k)]["surviving_probability"] = deepcopy(new_sp)
                
                # Try to find a new solution
                are_there_free_srlgs, new_srlgs = min_prob_rule(new_srlgs, number_of_backups=0)

                # Generate new input string and solve the problem
                input_string = generate_input_string(G_scaled, s, t, new_srlgs)
                special_path, walks, potentials, cutting_edge_groups = solve_from_string(exe_path, input_string)
                new_walks = shorten_paths(G, s, t, walks, new_srlgs)
                new_sp = get_surviving_probabilities_of_walks(G, new_walks)[k]

            print()
            short_solutions[graph_name]["terminals"][(s, t)]["min_cutting_prob"][(n, k)] = deepcopy(short_solution[(n, k)])
            path = os.path.join("results", f"short_solutions_{tolerance}.pkl")
            with open(path, "wb") as f_out:
                pickle.dump(short_solutions, f_out)
    end_time = time.time()
    print(f"Time taken: {end_time - start_time:.4f} seconds")

Processing graph: 22_optic_eu_latlon
	terminal 1:
		number of paths: 3
			batch iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 
			one by one iteration: 1, 2, 3, 4, 5, 
		number of paths: 4
			batch iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 
			one by one iteration: 1, 2, 3, 4, 5, 6, 7, 
		number of paths: 5
			batch iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 
			one by one iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 
		number of paths: 6
			batch iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 
			one by one iteration: 1, 2, 
		number of paths: 7
			batch iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 1

##### Tolerance: 1, backups: 3, only for "22_optic_eu_latlon"

runtime: 3h

In [49]:
exe_path = os.path.join("cpp_codes", "dateline_capacity.exe")
tolerance = 0.01

# Load the results
short_solutions = {}
path = os.path.join("results", "short_solutions_0.01.pkl")
with open(path, "rb") as f:
    short_solutions = pickle.load(f)

for graph_name in ["22_optic_eu_latlon"]:
    G = improved_solutions[graph_name]["graph"]
    G_scaled = G.copy()
    for node, data in G_scaled.nodes(data=True):
        G_scaled.nodes()[node]['pos'] = (round(data['pos'][0]*100), round(data['pos'][1]*100))
    start_time = time.time()
    print(f"Processing graph: {graph_name}")
    terminal_i = 0
    for s, t in improved_solutions[graph_name]["terminals"].keys():
        terminal_i += 1
        print(f"\tterminal {terminal_i}:")
        # get the corrected solution
        solution = improved_solutions[graph_name]["terminals"][(s, t)]["min_cutting_prob"]
        short_solution = deepcopy(solution)
        # go through each number of paths and backups
        for n in range(4, 11):
            print(f"\t\tnumber of paths: {n}")
            number_of_backups = 3
            k = n - number_of_backups

            # try to shorten the paths
            short_solution[(n, k)]["srlgs"] = make_srlgs_tight(G, solution[(n, k)]["walks"])
            new_srlgs = deepcopy(short_solution[(n, k)]["srlgs"])
            prev_srlgs = deepcopy(new_srlgs)
            short_solution[(n, k)]["walks"] = shorten_paths(G, s, t, solution[(n, k)]["walks"], new_srlgs)
            new_walks = deepcopy(short_solution[(n, k)]["walks"])
            short_solution[(n, k)]["surviving_probability"] = get_surviving_probabilities_of_walks(G, new_walks)[k]
            new_sp = deepcopy(short_solution[(n, k)]["surviving_probability"])
            original_sp = deepcopy(new_sp)

            # Increase SRLG capacities in batches
            batch_i = 0
            are_there_free_srlgs = True
            print(f"\t\t\tbatch iteration: ", end='')
            while (
                len(new_walks) == n and 
                np.log10(1-new_sp) <  np.log10(1-original_sp)*(1-tolerance) and
                are_there_free_srlgs
            ):
                prev_srlgs = deepcopy(new_srlgs)
                batch_i += 1
                print(batch_i, end=', ')
                old_len = sum(len(walk)-1 for walk in short_solution[(n, k)]["walks"])
                new_len = sum(len(walk)-1 for walk in new_walks)

                # If the new solution is better, save it
                if new_len < old_len:
                    short_solution[(n, k)]["srlgs"] = deepcopy(new_srlgs)
                    short_solution[(n, k)]["walks"] = deepcopy(new_walks)
                    short_solution[(n, k)]["surviving_probability"] = deepcopy(new_sp)

                # Try to find a new solution
                are_there_free_srlgs, new_srlgs = min_prob_batch_rule(new_srlgs, number_of_backups=number_of_backups)

                # Generate new input string and solve the problem
                input_string = generate_input_string(G_scaled, s, t, new_srlgs)
                special_path, walks, potentials, cutting_edge_groups = solve_from_string(exe_path, input_string)
                new_walks = shorten_paths(G, s, t, walks, new_srlgs)
                new_sp = get_surviving_probabilities_of_walks(G, new_walks)[k]
            print()

            # Get the last valid solution
            new_srlgs = deepcopy(prev_srlgs)
            input_string = generate_input_string(G_scaled, s, t, new_srlgs)
            special_path, walks, potentials, cutting_edge_groups = solve_from_string(exe_path, input_string)
            new_walks = shorten_paths(G, s, t, walks, new_srlgs)
            new_sp = get_surviving_probabilities_of_walks(G, new_walks)[k]

            # Increase SRLG capacities one by one
            one_by_one_i = 0
            are_there_free_srlgs = True
            print(f"\t\t\tone by one iteration: ", end='')
            while (
                len(new_walks) == n and 
                np.log10(1-new_sp) <  np.log10(1-original_sp)*(1-tolerance) and
                are_there_free_srlgs 
            ):
                one_by_one_i += 1
                print(one_by_one_i, end=', ')
                old_len = sum(len(walk)-1 for walk in short_solution[(n, k)]["walks"])
                new_len = sum(len(walk)-1 for walk in new_walks)

                # If the new solution is better, save it
                if new_len < old_len:
                    short_solution[(n, k)]["srlgs"] = deepcopy(new_srlgs)
                    short_solution[(n, k)]["walks"] = deepcopy(new_walks)
                    short_solution[(n, k)]["surviving_probability"] = deepcopy(new_sp)
                
                # Try to find a new solution
                are_there_free_srlgs, new_srlgs = min_prob_rule(new_srlgs, number_of_backups=0)

                # Generate new input string and solve the problem
                input_string = generate_input_string(G_scaled, s, t, new_srlgs)
                special_path, walks, potentials, cutting_edge_groups = solve_from_string(exe_path, input_string)
                new_walks = shorten_paths(G, s, t, walks, new_srlgs)
                new_sp = get_surviving_probabilities_of_walks(G, new_walks)[k]

            print()
            short_solutions[graph_name]["terminals"][(s, t)]["min_cutting_prob"][(n, k)] = deepcopy(short_solution[(n, k)])
            path = os.path.join("results", f"short_solutions_{tolerance}.pkl")
            with open(path, "wb") as f_out:
                pickle.dump(short_solutions, f_out)
    end_time = time.time()
    print(f"Time taken: {end_time - start_time:.4f} seconds")

Processing graph: 22_optic_eu_latlon
	terminal 1:
		number of paths: 4
			batch iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 
			one by one iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 
		number of paths: 5
			batch iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 
			one by one iteration: 1, 2, 3, 
		number of paths: 6
			batch iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 
			one by one iteration: 1, 2, 3, 4, 
		number of paths: 7
			batch iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 2

### With trimming

#### Initial Solutions

In [10]:
# runtime: 13 min

# path to the executable, which solves the SRLG problem
exe_path = os.path.join("cpp_codes", "dateline_prob.exe")

# Load the results
initial_solutions = {}
path = os.path.join("results", "initial_solutions.pkl")
with open(path, "rb") as f:
    initial_solutions = pickle.load(f)

removed_edges_test = {graph_name: {
    "graph": initial_solutions[graph_name]["graph"],
    "terminals": { (s, t): {
        d: {} for d in range(5)
    } for (s, t) in initial_solutions[graph_name]["terminals"].keys() }
} for graph_name in initial_solutions.keys() }

for d in range(5):
    print(f"d: {d}")
    for graph_name in initial_solutions.keys():
        print(f"\tgraph name: {graph_name}", end=': ')
        G = initial_solutions[graph_name]["graph"]
        skipped_graphs = 0

        # For all terminal pairs
        for i, (s, t) in enumerate(initial_solutions[graph_name]["terminals"].keys()):
            # print(f"\tFinding paths for source: {s} ({G.nodes[s]['label']}), target:, {t} ({G.nodes[t]['label']})")
            print(f"{i}", end=', ')

            distance = nx.shortest_path_length(G, source=s, target=t)
            # Create a copy of the graph with high-cost edges removed
            H = remove_high_cost_edges(G, s, t, distance+d, remove_nodes=True)

            if nx.is_forest(H):
                skipped_graphs += 1
                #print(f"\t\tskipped")
                continue

            solution = removed_edges_test[graph_name]["terminals"][(s, t)][d]["min_cutting_prob"] = {}

            solution['graph'] = H
            solution['surviving_probabilities'] = []
            solution['walks'] = []
            solution['srlgs'] = []

            # Set all capacities to 1
            for srlg in H.graph['srlgs']:
                srlg['capacity'] = 1

            # C++ code needs integer coordinates, so scale positions
            H_scaled = H.copy()
            for node, data in H_scaled.nodes(data=True):
                H_scaled.nodes()[node]['pos'] = (round(data['pos'][0]*100), round(data['pos'][1]*100))

            # Solve
            input_string = generate_input_string_with_prob(H_scaled, s, t)
            number_of_backups = 0
            if method[-1].isdigit():
                number_of_backups = int(method[-1])
                method = method[:-2]
            output = solve_from_string_prob(exe_path, input_string, method = method, number_of_backups=number_of_backups)
            for sol_dict in output.values():
                solution['walks'].append(sol_dict['paths'])
                solution['surviving_probabilities'].append(get_surviving_probabilities_of_walks(H, solution["walks"][-1]))
                solution['srlgs'].append(deepcopy(H.graph["srlgs"]))
                for srlg_id, srlg_capacity in sol_dict['srlgs'].items():
                    for srlg in solution["srlgs"][-1]:
                        if srlg['id'] == srlg_id:
                            srlg['capacity'] = srlg_capacity

        print()

    # save the results into pickle file
    path = os.path.join("results", "trimmed_initial_solutions.pkl")
    with open(path, "wb") as f:
        pickle.dump(removed_edges_test, f)

d: 0
	graph name: 22_optic_eu_latlon: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 
	graph name: 26_usanet_latlon: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 
	graph name: 28_optic_eu_latlon: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 
	graph name: 37_eu_latlon: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 
	graph name: 39_optic_north_american_latlon: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 

#### Improve Solutions

In [11]:
# runtime: 10 min
# Load the results
initial_solutions = {}
path = os.path.join("results", "trimmed_initial_solutions.pkl")
with open(path, "rb") as f:
    initial_solutions = pickle.load(f)

fixed_solutions = deepcopy(initial_solutions)

for d in range(5):
    print(f"d: {d}")
    for graph_name in initial_solutions.keys():
        print(f"\t{graph_name}", end=': ')
        G = initial_solutions[graph_name]["graph"]

        for i, (s, t) in enumerate(initial_solutions[graph_name]["terminals"].keys()):
            print(i, end=", ")
            solution = initial_solutions[graph_name]["terminals"][(s, t)][d]
            if solution == {}:
                continue
            else:
                solution = solution[method]
                H = solution["graph"]

                improved_solution = improve_all_surviving_probabilities(H, solution)
                improved_solution["graph"] = H
                fixed_solutions[graph_name]["terminals"][(s, t)][d][method] = improved_solution
        print()

    # save the results into pickle file
    path = os.path.join("results", "trimmed_improved_solutions.pkl")
    with open(path, "wb") as f:
        pickle.dump(fixed_solutions, f)

d: 0
	22_optic_eu_latlon: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 
	26_usanet_latlon: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 
	28_optic_eu_latlon: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 
	37_eu_latlon: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 
	39_optic_north_american_latlon: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 

#### Shorten Paths

only for 1 backup path

In [13]:
# runtime: 2 min
# Load the results
fixed_solutions = {}
path = os.path.join("results", "trimmed_improved_solutions.pkl")
with open(path, "rb") as f:
    fixed_solutions = pickle.load(f)

short_solutions = deepcopy(fixed_solutions)

for d in range(5):
    print(f"d: {d}")
    for graph_name in fixed_solutions.keys():
        print(f"\t{graph_name}", end=': ')
        G = fixed_solutions[graph_name]["graph"]

        for i, (s, t) in enumerate(fixed_solutions[graph_name]["terminals"].keys()):
            print(i, end=", ")
            solution = fixed_solutions[graph_name]["terminals"][(s, t)][d]
            if solution == {}:
                continue
            else:
                solution = solution[method]
                H = solution["graph"]
                shortened_solution = deepcopy(solution)

                for n in range(2, 11):

                    og_paths = solution[(n, n-1)]["walks"]
                    og_srlgs = solution[(n, n-1)]["srlgs"]
                    H = solution["graph"]
                    short_paths = shorten_paths(H, s, t, og_paths, og_srlgs)

                    og_len = sum(len(walk) - 1 for walk in og_paths)
                    short_len = sum(len(walk) - 1 for walk in short_paths)
                    if short_len < og_len:
                        shortened_solution[(n, n-1)]["walks"] = short_paths
                        shortened_solution[(n, n-1)]["surviving_probability"] = get_surviving_probabilities_of_walks(H, short_paths)[n-1]

                short_solutions[graph_name]["terminals"][(s, t)][d][method] = shortened_solution
        print()

    # save the results into pickle file
    path = os.path.join("results", "trimmed_shortened_solutions.pkl")
    with open(path, "wb") as f:
        pickle.dump(short_solutions, f)

d: 0
	22_optic_eu_latlon: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 
	26_usanet_latlon: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 
	28_optic_eu_latlon: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 
	37_eu_latlon: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 
	39_optic_north_american_latlon: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 

#### `min_cutting_prob`

##### Helper Functions

In [17]:
def min_prob_rule(srlgs, number_of_backups=0, max_number_of_paths=10):
    new_srlgs = deepcopy(srlgs)
    eligible_srlgs = [srlg for srlg in new_srlgs if srlg['capacity'] < max_number_of_paths]
    # RULE: Find the SRLG with the minimum probability * capacity
    if not eligible_srlgs:
            return False, new_srlgs
    min_probability_srlg = min(eligible_srlgs, key=lambda srlg: srlg['probability'] * srlg['capacity'])
    min_probability_srlg["capacity"] += 1
    if number_of_backups > 0 and min_probability_srlg["capacity"] > number_of_backups:
        min_probability_srlg["capacity"] = max_number_of_paths
    return True, new_srlgs

def min_prob_batch_rule(srlgs, batch_ratio = 0.05, number_of_backups=0, max_number_of_paths=10):
    # RULE: Find the SRLG with the minimum probability * capacity
    new_srlgs = deepcopy(srlgs)
    for _ in range(max(2, int(len(new_srlgs) * batch_ratio))):
        eligible_srlgs = [srlg for srlg in new_srlgs if srlg['capacity'] < max_number_of_paths]
        if not eligible_srlgs:
            return False, new_srlgs
        min_probability_srlg = min(eligible_srlgs, key=lambda srlg: srlg['probability'] * srlg['capacity'])
        min_probability_srlg["capacity"] += 1
        if number_of_backups > 0 and min_probability_srlg["capacity"] > number_of_backups:
            min_probability_srlg["capacity"] = max_number_of_paths
    return True, new_srlgs

##### Tolerance: 1%, backups: 1

In [19]:
# Load the results
fixed_solutions = {}
path = os.path.join("results", "trimmed_shortened_solutions.pkl")
with open(path, "rb") as f:
    fixed_solutions = pickle.load(f)

exe_path = os.path.join("cpp_codes", "dateline_capacity.exe")
tolerance = 0.01
short_solutions = deepcopy(fixed_solutions)

for d in range(5):
    print(f"d: {d}")
    for graph_name in fixed_solutions.keys():
        print(f"\t{graph_name}")
        G = fixed_solutions[graph_name]["graph"]
        start_time = time.time()
        for i, (s, t) in enumerate(fixed_solutions[graph_name]["terminals"].keys()):
            print(f"\t\tterminal {i}")
            solution = fixed_solutions[graph_name]["terminals"][(s, t)][d]
            if solution == {}:
                continue
            else:
                solution = solution["min_cutting_prob"]
                H = solution["graph"]
                H_scaled = H.copy()
                for node, data in H_scaled.nodes(data=True):
                    H_scaled.nodes()[node]['pos'] = (round(data['pos'][0]*100), round(data['pos'][1]*100))
                short_solution = deepcopy(solution)

                for n in range(2, 11):
                    print(f"\t\t\tnumber of paths: {n}")
                    # costants for this testing environment
                    number_of_backups = 1
                    k = n-number_of_backups

                    # try to shorten the paths
                    short_solution[(n, k)]["srlgs"] = make_srlgs_tight(H, solution[(n, k)]["walks"])
                    new_srlgs = deepcopy(short_solution[(n, k)]["srlgs"])
                    prev_srlgs = deepcopy(new_srlgs)
                    short_solution[(n, k)]["walks"] = shorten_paths(H, s, t, solution[(n, k)]["walks"], new_srlgs)
                    new_walks = deepcopy(short_solution[(n, k)]["walks"])
                    short_solution[(n, k)]["surviving_probability"] = get_surviving_probabilities_of_walks(H, new_walks)[k]
                    new_sp = deepcopy(short_solution[(n, k)]["surviving_probability"])
                    original_sp = deepcopy(new_sp)

                    # Increase SRLG capacities in batches
                    batch_i = 0
                    are_there_free_srlgs = True
                    print(f"\t\t\t\tbatch iteration: ", end='')
                    while (
                        len(new_walks) == n and 
                        np.log10(1-new_sp) <  np.log10(1-original_sp)*(1-tolerance) and
                        are_there_free_srlgs
                    ):
                        prev_srlgs = deepcopy(new_srlgs)
                        batch_i += 1
                        print(batch_i, end=', ')
                        old_len = sum(len(walk)-1 for walk in short_solution[(n, k)]["walks"])
                        new_len = sum(len(walk)-1 for walk in new_walks)

                        # If the new solution is better, save it
                        if new_len < old_len:
                            short_solution[(n, k)]["srlgs"] = deepcopy(new_srlgs)
                            short_solution[(n, k)]["walks"] = deepcopy(new_walks)
                            short_solution[(n, k)]["surviving_probability"] = deepcopy(new_sp)

                        # Try to find a new solution
                        are_there_free_srlgs, new_srlgs = min_prob_batch_rule(new_srlgs, number_of_backups=number_of_backups)

                        # Generate new input string and solve the problem
                        input_string = generate_input_string(H_scaled, s, t, new_srlgs)
                        special_path, walks, potentials, cutting_edge_groups = solve_from_string(exe_path, input_string)
                        new_walks = shorten_paths(H, s, t, walks, new_srlgs)
                        new_sp = get_surviving_probabilities_of_walks(H, new_walks)[k]
                    print()

                    # Get the last valid solution
                    new_srlgs = deepcopy(prev_srlgs)
                    input_string = generate_input_string(H_scaled, s, t, new_srlgs)
                    special_path, walks, potentials, cutting_edge_groups = solve_from_string(exe_path, input_string)
                    new_walks = shorten_paths(H, s, t, walks, new_srlgs)
                    new_sp = get_surviving_probabilities_of_walks(H, new_walks)[k]

                    # Increase SRLG capacities one by one
                    one_by_one_i = 0
                    print(f"\t\t\t\tone by one iteration: ", end='')
                    are_there_free_srlgs = True
                    while (
                        len(new_walks) == n and 
                        np.log10(1-new_sp) <  np.log10(1-original_sp)*(1-tolerance) and
                        are_there_free_srlgs
                    ):
                        one_by_one_i += 1
                        print(one_by_one_i, end=', ')
                        old_len = sum(len(walk)-1 for walk in short_solution[(n, k)]["walks"])
                        new_len = sum(len(walk)-1 for walk in new_walks)

                        # If the new solution is better, save it
                        if new_len < old_len:
                            short_solution[(n, k)]["srlgs"] = deepcopy(new_srlgs)
                            short_solution[(n, k)]["walks"] = deepcopy(new_walks)
                            short_solution[(n, k)]["surviving_probability"] = deepcopy(new_sp)
                        
                        # Try to find a new solution
                        are_there_free_srlgs, new_srlgs = min_prob_rule(new_srlgs, number_of_backups=0)

                        # Generate new input string and solve the problem
                        input_string = generate_input_string(H_scaled, s, t, new_srlgs)
                        special_path, walks, potentials, cutting_edge_groups = solve_from_string(exe_path, input_string)
                        new_walks = shorten_paths(H, s, t, walks, new_srlgs)
                        new_sp = get_surviving_probabilities_of_walks(H, new_walks)[k]

                    print()
                    short_solutions[graph_name]["terminals"][(s, t)][d]["min_cutting_prob"][(n, k)] = deepcopy(short_solution[(n, k)])
                    
                    path = os.path.join("results", f"trimmed_short_solutions_{tolerance}.pkl")
                    with open(path, "wb") as f_out:
                        pickle.dump(short_solutions, f_out)

        end_time = time.time()
        print(f"Time taken: {end_time - start_time:.4f} seconds")
        print()

d: 0
	22_optic_eu_latlon
		terminal 0
		terminal 1
		terminal 2
		terminal 3
		terminal 4
			number of paths: 2
				batch iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 
				one by one iteration: 1, 
			number of paths: 3
				batch iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 
				one by one iteration: 1, 
			number of paths: 4
				batch iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 
				one by one iteration: 1, 2, 
			number of paths: 5
				batch iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 
				one by one iteration: 1, 
			number of paths: 6
				batch iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 
				one by one iteration: 1, 2, 3, 
			number of paths: 7
				batch iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 
				one by one iteration: 1, 
			number of paths: 8
				batch iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 
				one by one iteration: 1, 
			number of paths: 9
				batch iteration: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 
				one by one iteration: 1, 
			number of paths: 10
				batch iteration: 1

## Results

### Compare 1-backup solutions with 2 paths

itt azt látjuk, hogy a mi megoldásaink is mindig belsőleg csúcs-diszjunktak

In [25]:
default_colors = px.colors.qualitative.Plotly

# Load the results
short_solutions = {}
path = os.path.join("results", "short_solutions_0.01.pkl")
with open(path, "rb") as f:
    short_solutions = pickle.load(f)
    
figures = []
# Test parameters
for graph_name in graph_names:
    print(graph_name)
    G = short_solutions[graph_name]["graph"]
    method = "min_cutting_prob"

    for terminal_index, (s, t) in enumerate(short_solutions[graph_name]["terminals"].keys()):
        # Our solution
        solution = short_solutions[graph_name]["terminals"][(s, t)][method][2, 1]
        walks = solution["walks"]

        # Disjoint paths
        disjoint_paths = list(nx.node_disjoint_paths(G, s, t, cutoff=2))

        # Get SRLGs intersecting the disjoint paths
        edges_of_disjoint_paths = make_walks_by_edges(G, disjoint_paths)
        disjoint_paths_edge_ids = set(eid for path in edges_of_disjoint_paths for eid in path)
        intersecting_srlgs = [
            srlg for srlg in G.graph['srlgs']
            if set(srlg['edges']) & disjoint_paths_edge_ids
        ]

        # Top SRLGs
        top_srlgs = sorted(intersecting_srlgs, key=lambda s: s['probability'], reverse=True)[:4]
        H = planar_embed(G)
        srlg_traces = get_srlg_traces(H, top_srlgs, dash='dot')
        for idx, trace in enumerate(srlg_traces):
            trace.line.color = default_colors[2 + idx]
            trace.showlegend = (idx == 0)
            trace.name = 'srlgs'

        # Path traces
        traces = get_path_traces(G, disjoint_paths + walks)

        # Disjoint paths
        disjoint_path_traces = traces[:2]
        for i, trace in enumerate(disjoint_path_traces):
            trace.showlegend = (i == 0)
            trace.name = "node disjoint paths" if i == 0 else None
            trace.line.color = default_colors[0]
            if i == 1:
                trace.line.dash = "dash"

        # Our solution
        solution_trace = traces[2:]
        for i, trace in enumerate(solution_trace):
            trace.showlegend = (i == 0)
            trace.name = "our solution" if i == 0 else None
            trace.line.color = default_colors[1]
            if i == 1:
                trace.line.dash = "dash"

        # Base graph
        edge_trace = get_edges_trace(G)
        node_trace = get_nodes_trace(G, label='id')

        # Change the colors of node s and t
        node_colors = ['lightblue'] * len(node_trace.x)
        node_labels = list(node_trace.text)
        s_index = node_labels.index(str(s))
        t_index = node_labels.index(str(t))
        node_colors[s_index] = 'lightpink'
        node_colors[t_index] = 'lightpink'
        node_trace.marker.color = node_colors

        # Combine everything
        traces = disjoint_path_traces + solution_trace + [edge_trace, node_trace] + srlg_traces

        fig = go.Figure(
            data=traces,
            layout=go.Layout(
                margin=dict(b=5, l=5, r=5, t=30),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False, scaleanchor="x"),
                plot_bgcolor='white',
                paper_bgcolor='white'
            )
        )

        fig.update_layout(
            title=f"{graph_name} {terminal_index}: ({s}, {t})",
            legend=dict(
                x=0.01,
                y=0.99,
                bgcolor='rgba(255,255,255,0.8)',
                bordercolor='black',
                borderwidth=1,
                font=dict(size=12),
            ),
        )

        conflicts = check_pairwise_internal_intersections(walks)
        if conflicts:
            print("There are common nodes:")
            for (i, j), nodes in conflicts:
                print(f"  path {i} and path {j} has common nodes: {nodes} at terminal pair {terminal_index}, ({s}, {t})")
            figures.append(fig)


# Dash app
app = Dash()
app.layout = html.Div([
    html.Div([
        html.Div(
            dcc.Graph(figure=figures[i]),
            style={'width': '48%', 'display': 'inline-block', 'padding': '10px'}
        )
        for i in range(len(figures))
    ])
])
app.run()


22_optic_eu_latlon
25_interoute_latlon
There are common nodes:
  path 0 and path 1 has common nodes: {17} at terminal pair 0, (9, 23)
There are common nodes:
  path 0 and path 1 has common nodes: {17} at terminal pair 2, (21, 23)
There are common nodes:
  path 0 and path 1 has common nodes: {17} at terminal pair 8, (18, 23)
There are common nodes:
  path 0 and path 1 has common nodes: {17} at terminal pair 21, (12, 23)
There are common nodes:
  path 0 and path 1 has common nodes: {17} at terminal pair 40, (11, 23)
26_usanet_latlon
28_optic_eu_latlon
37_eu_latlon
39_optic_north_american_latlon
79_optic_nfsnet_latlon
There are common nodes:
  path 0 and path 1 has common nodes: {5} at terminal pair 9, (4, 64)
There are common nodes:
  path 0 and path 1 has common nodes: {30} at terminal pair 12, (4, 29)
There are common nodes:
  path 0 and path 1 has common nodes: {30} at terminal pair 13, (19, 29)
There are common nodes:
  path 0 and path 1 has common nodes: {30} at terminal pair 46, (1

### Compare 1-backup solutions with 3 paths

Itt már azt látjuk, hogy egy esetben, a 8 és 14 csúcsok között az általunk talált utaknak van egy közös belső pontja

In [5]:
def check_pairwise_internal_intersections(walks):
    """
    walks: lista, utak listája, pl. [[s,...,t], [...], ...]
    Visszaad:
      - conflicts: lista ( (i,j), conflict_nodes ) alakban
        ahol i < j, és conflict_nodes nem üres metszet
    """
    # belső csúcsok kiszedése minden útra
    internal_sets = [
        set(walk[1:-1]) for walk in walks
    ]

    conflicts = []

    # páronkénti összehasonlítás
    n = len(internal_sets)
    for i in range(n):
        for j in range(i + 1, n):
            common = internal_sets[i] & internal_sets[j]
            if common:
                conflicts.append(((i, j), common))

    return conflicts


In [20]:
default_colors = px.colors.qualitative.Plotly

# Load the results
short_solutions = {}
path = os.path.join("results", "short_solutions_0.01.pkl")
with open(path, "rb") as f:
    short_solutions = pickle.load(f)
    
# Test parameters
figures = []
for graph_name in graph_names:
    print(graph_name)
    G = short_solutions[graph_name]["graph"]
    method = "min_cutting_prob"

    for terminal_index, (s, t) in enumerate(short_solutions[graph_name]["terminals"].keys()):
        # Our solution
        solution = short_solutions[graph_name]["terminals"][(s, t)][method][3, 2]
        walks = solution["walks"]


        # Disjoint paths
        disjoint_paths = list(nx.node_disjoint_paths(G, s, t, cutoff=3))
        if len(disjoint_paths) < 3:
            continue
        conflicts = check_pairwise_internal_intersections(walks)
        if conflicts:
            print("There are common nodes:")
            for (i, j), nodes in conflicts:
                print(f"  path {i} and path {j} has common nodes: {nodes} at terminal pair {terminal_index}, ({s}, {t})")

        # Get SRLGs intersecting the disjoint paths
        edges_of_disjoint_paths = make_walks_by_edges(G, disjoint_paths)
        disjoint_paths_edge_ids = set(eid for path in edges_of_disjoint_paths for eid in path)
        intersecting_srlgs = [
            srlg for srlg in G.graph['srlgs']
            if set(srlg['edges']) & disjoint_paths_edge_ids
        ]

        # Top SRLGs
        top_srlgs = sorted(intersecting_srlgs, key=lambda s: s['probability'], reverse=True)[:min(len(intersecting_srlgs), 5)]
        H = planar_embed(G)
        srlg_traces = get_srlg_traces(H, top_srlgs, dash='dot')
        for idx, trace in enumerate(srlg_traces):
            trace.line.color = default_colors[2 + idx]
            trace.showlegend = (idx == 0)
            trace.name = 'srlgs'

        # Path traces
        traces = get_path_traces(G, disjoint_paths + walks)

        # Disjoint paths
        disjoint_path_traces = traces[:3]
        for i, trace in enumerate(disjoint_path_traces):
            trace.showlegend = (i == 0)
            trace.name = "node disjoint paths" if i == 0 else None
            trace.line.color = default_colors[0]
            if i == 1:
                trace.line.dash = "dash"
            elif i == 2:
                trace.line.dash = "dashdot"

        # Our solution
        solution_trace = traces[3:]
        for i, trace in enumerate(solution_trace):
            trace.showlegend = (i == 0)
            trace.name = "our solution" if i == 0 else None
            trace.line.color = default_colors[1]
            if i == 1:
                trace.line.dash = "dash"
            elif i == 2:
                trace.line.dash = "dashdot"

        # Base graph
        edge_trace = get_edges_trace(G)
        node_trace = get_nodes_trace(G, label='id')

        # Change the colors of node s and t
        node_colors = ['lightblue'] * len(node_trace.x)
        node_labels = list(node_trace.text)
        s_index = node_labels.index(str(s))
        t_index = node_labels.index(str(t))
        node_colors[s_index] = 'lightpink'
        node_colors[t_index] = 'lightpink'
        node_trace.marker.color = node_colors

        # Combine everything
        traces = disjoint_path_traces + solution_trace + [edge_trace, node_trace] + srlg_traces

        fig = go.Figure(
            data=traces,
            layout=go.Layout(
                margin=dict(b=5, l=5, r=5, t=30),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False, scaleanchor="x"),
                plot_bgcolor='white',
                paper_bgcolor='white'
            )
        )

        fig.update_layout(
            title=f"{graph_name} {terminal_index}: ({s}, {t})",
            legend=dict(
                x=0.01,
                y=0.99,
                bgcolor='rgba(255,255,255,0.8)',
                bordercolor='black',
                borderwidth=1,
                font=dict(size=12),
            ),
        )

        figures.append(fig)

# Dash app
app = Dash()
app.layout = html.Div([
    html.Div([
        html.Div(
            dcc.Graph(figure=figures[i]),
            style={'width': '48%', 'display': 'inline-block', 'padding': '10px'}
        )
        for i in range(len(figures))
    ])
])
app.run()


22_optic_eu_latlon
There are common nodes:
  path 0 and path 2 has common nodes: {15} at terminal pair 42, (8, 14)
25_interoute_latlon
26_usanet_latlon
28_optic_eu_latlon
37_eu_latlon
There are common nodes:
  path 1 and path 2 has common nodes: {24} at terminal pair 22, (22, 32)
39_optic_north_american_latlon
79_optic_nfsnet_latlon


### Compare number of backups

itt azt látjuk, hogy szinte mindig jobban megéri 1 backupot csinálni. alkalmanként amikor 10 utat keresünk, akkor annak lehet kisebb a sávszélessége.
jó példák, hogy az 1 backup jobb: 
(4, 5)
(4, 11)
(1, 11)
(6, 19)
   (5, 9)
(3, 11)
(10, 16)
(7, 11)


In [14]:
def create_failure_magnitudes_improved(G, solution, title, max_backups=3, max_number_of_walks=10, original_G=None):
    fig = go.Figure()
              
    for backups in range(1, max_backups+1):
        x = []
        y = []
        text = []

        # calculate data to plot
        for n in range(backups+1, max_number_of_walks+1):
            x.append(np.log10(1-solution[(n, n-backups)]['surviving_probability']))
            total_length = sum(len(walk) - 1 for walk in solution[(n,n-backups)]['walks'])
            bandwidth = total_length / (n - backups)
            y.append(bandwidth)
            text.append(f"{n}")

        # add the data to the figure
        fig.add_trace(go.Scatter(
            x = deepcopy(x),
            y = deepcopy(y),
            mode='lines+markers+text',
            name=f"{backups} backups",
            text = deepcopy(text),
            textposition='top center',
        ))

    # Add edge disjoint paths
    # if original_G is not None:
    #     G = original_G
    # H = deepcopy(G)
    # default_colors = px.colors.qualitative.Plotly
    # for edge in H.edges(data=True): 
    #     edge[2]['capacity'] = 1
    # edge_disjoint_paths = [path for path in nx.edge_disjoint_paths(H, s, t)]
    # for backups in range(1, max_backups+1):
    #     if len(edge_disjoint_paths) > backups:
    #         total_length = sum(len(disjoint_path) - 1 for disjoint_path in edge_disjoint_paths)
    #         bandwidth = total_length / (len(edge_disjoint_paths) - backups)
    #         fig.add_trace(go.Scatter(
    #             x=[np.log10(1-get_surviving_probabilities_of_walks(H, edge_disjoint_paths)[-1-backups])],
    #             y=[bandwidth],
    #             mode='markers',
    #             marker = dict(color=default_colors[backups-1], symbol='square', size = 8),
    #             #name=f'{len(disjoint_paths)} disjoint paths with {backups} backups',
    #             name=f'{len(edge_disjoint_paths)} edge disjoint paths',
    #             legendgroup=f'{len(edge_disjoint_paths)} edge disjoint paths',
    #             showlegend=(backups == 1),  # only show legend for the first backup
    #         ))

    # add node disjoint paths
    # node_disjoint_paths = list(nx.node_disjoint_paths(G, s, t))
    # for backups in range(1, max_backups+1):
    #     if len(node_disjoint_paths) > backups:
    #         total_length = sum(len(disjoint_path) - 1 for disjoint_path in node_disjoint_paths)
    #         bandwidth = total_length / (len(node_disjoint_paths) - backups)
    #         fig.add_trace(go.Scatter(
    #             x=[np.log10(1-get_surviving_probabilities_of_walks(H, node_disjoint_paths)[-1-backups])],
    #             y=[bandwidth],
    #             mode='markers',
    #             marker = dict(color=default_colors[backups-1], symbol='diamond', size=8),
    #             #name=f'{len(disjoint_paths)} disjoint paths with {backups} backups',
    #             name=f'{len(node_disjoint_paths)} node disjoint paths',
    #             legendgroup=f'{len(node_disjoint_paths)} node disjoint paths',
    #             showlegend=(backups == 1),  # only show legend for the first backup
    #         ))

    #add the shortest path
    shortest_path = nx.shortest_path(G, source=s, target=t)
    shortest_path_sp = get_surviving_probabilities_of_walks(G, [shortest_path])[-1]
    fig.add_trace(go.Scatter(
        x=[np.log10(1-shortest_path_sp)],
        y=[len(shortest_path)-1],
        mode='markers',
        marker = dict(color='black', symbol='x', size=8),
        name=f'Shortest path',
    ))


    # update the layout of the figure
    fig.update_layout(
        title=title,
        xaxis=dict(
            title='Failure Probability Magnitude',
        ),
        yaxis = dict(
            title='Bandwidth',
           # dtick = 1.
        ),
    )
    return fig

solutions = {}
with open(os.path.join("results", "short_solutions_0.01.pkl"), "rb") as f:
    solutions = pickle.load(f)

graph_name = "22_optic_eu_latlon"
G = solutions[graph_name]["graph"]
method = "min_cutting_prob"
# s, t = 4, 17

# solution = solutions[graph_name]["terminals"][(s, t)][method]
# fig = create_failure_magnitudes_improved(G, solutions[graph_name]["terminals"][(s, t)][method], f"Failure magnitude and bandwidth for {method} on {graph_name} <br> with terminals ({s}, {t}), distance: {nx.shortest_path_length(G, source=s, target=t)}, tolerance: 1%")
# fig.show()

# # calculate data to plot
# for backups in range(1, 4):
#     print("x y label")
#     for n in range(backups+1, 11):
#         prob = 1-solution[(n, n-backups)]['surviving_probability']
#         bandwidth = sum(len(walk) - 1 for walk in solution[(n,n-backups)]['walks']) / (n - backups)
#         text = f"{n}"
#         print(f"{prob:.6f} {bandwidth:.2f} {text}")
#     print()

# shortest_path = nx.shortest_path(G, source=s, target=t)
# shortest_path_sp = get_surviving_probabilities_of_walks(G, [shortest_path])[-1]
# #print(f"{1-shortest_path_sp:.6f}")

# cutting_srlgs = []
# for srlg in G.graph["srlgs"]:
#     # Create a copy of the graph to avoid modifying the original
#     G_copy = G.copy()
#     # Remove the edges in the current SRLG
#     for edge_id in srlg["edges"]:
#         u, v = G.graph["edge_by_id"][edge_id]
#         G_copy.remove_edge(u, v)
#     # Check if s_id and t_id are still connected
#     if not nx.has_path(G_copy, s, t):
#         cutting_srlgs.append(srlg)

# min_prob = sum(srlg["probability"] for srlg in cutting_srlgs)
# print(f"{min_prob:.6f}")


figures = []
for s, t in list(solutions[graph_name]["terminals"].keys()):
    fig = create_failure_magnitudes_improved(G, solutions[graph_name]["terminals"][(s, t)][method], f"Failure magnitude and bandwidth for {method} on {graph_name} <br> with terminals ({s}, {t}), distance: {nx.shortest_path_length(G, source=s, target=t)}, tolerance: 1%")
    figures.append(fig)
# dash app to display the figures
app = Dash()
# Arrange them in two columns
app.layout = html.Div([
    html.Div([
        html.Div(dcc.Graph(figure=figures[i]), style={'width': '48%', 'display': 'inline-block', 'padding': '10px'})
        for i in range(len(figures))
    ])
])
app.run()  # Turn off reloader if inside Jupyter Notebook

save_figures_to_html(figures, "backup_comparison.html", "Comparing Number of Backups")

KeyError: (3, 1)

### All comparison

In [77]:
def suurballe_node_disjoint(G: nx.Graph, s, t):
    G_split = nx.DiGraph()

    # Step 1: Split only intermediate nodes
    for v in G.nodes:
        if v not in (s, t):
            G_split.add_edge(f"{v}_in", f"{v}_out", weight=0)

    # Step 2: Replace edges appropriately
    for u, v, data in G.edges(data=True):
        w = data['weight']

        def map_in(x):
            return f"{x}_in" if x not in (s, t) else x

        def map_out(x):
            return f"{x}_out" if x not in (s, t) else x

        G_split.add_edge(map_out(u), map_in(v), weight=w)
        G_split.add_edge(map_out(v), map_in(u), weight=w)

    s_mod = s
    t_mod = t

    # Step 3: First shortest path
    dist, paths = nx.single_source_dijkstra(G_split, s_mod, weight='weight')
    if t_mod not in paths:
        raise nx.NetworkXNoPath(f"No path from {s} to {t}")
    path1 = paths[t_mod]

    # Step 4: Build reduced cost graph
    G_reduced = nx.DiGraph()
    for u, v, data in G_split.edges(data=True):
        if u in dist and v in dist:
            new_w = data['weight'] + dist[u] - dist[v]
            G_reduced.add_edge(u, v, weight=new_w, original_weight=data['weight'])

    # Step 5: Remove path1 edges and reverse them
    path_edges = list(zip(path1[:-1], path1[1:]))
    for u, v in path_edges:
        if G_reduced.has_edge(u, v):
            G_reduced.remove_edge(u, v)
        G_reduced.add_edge(v, u, weight=0)

    # Step 6: Second shortest path
    try:
        path2 = nx.shortest_path(G_reduced, s_mod, t_mod, weight='weight')
    except nx.NetworkXNoPath:
        raise nx.NetworkXNoPath("No second disjoint path found")

    # Step 7: Combine edges and eliminate overlaps
    edge_set = set()
    for u, v in zip(path1[:-1], path1[1:]):
        edge_set.add((u, v))
    for u, v in zip(path2[:-1], path2[1:]):
        if (v, u) in edge_set:
            edge_set.remove((v, u))
        else:
            edge_set.add((u, v))

    # Step 8: Build merged graph
    merged = nx.DiGraph()
    for u, v in edge_set:
        merged.add_edge(u, v)

    # Step 9: Extract disjoint paths
    paths = list(nx.edge_disjoint_paths(merged, s_mod, t_mod))
    if len(paths) < 2:
        raise RuntimeError("Failed to extract two disjoint paths")

    # Step 10: Convert back to original node labels
    def unsplit(path):
        result = []
        for node in path:
            node_str = str(node)
            if node_str.endswith('_in') or node_str.endswith('_out'):
                base = int(node_str.split('_')[0])
                if not result or result[-1] != base:
                    result.append(base)
            elif isinstance(node, int):
                if not result or result[-1] != node:
                    result.append(node)
        return result


    return unsplit(paths[0]), unsplit(paths[1])


Exception ignored in: 'zmq.backend.cython._zmq.Frame.__dealloc__'
Traceback (most recent call last):
  File "_zmq.py", line 169, in zmq.backend.cython._zmq._check_rc
KeyboardInterrupt: 


#### Shortest

In [79]:
graph_to_check = "79_optic_nfsnet_latlon"
terminals_to_check = [(4, 64), (4, 29), (19, 29), (16, 29)]

In [95]:
# Read test data
path = os.path.join("results", "short_solutions_0.01.pkl")
method = "min_cutting_prob"
solutions = {}
with open(path, "rb") as f:
    solutions = pickle.load(f)

# Make pandas dataframe
df = pd.DataFrame({
    "graph_name": pd.Series(dtype = str),
    "number_of_nodes": pd.Series(dtype="Int64"),
    "number_of_edges": pd.Series(dtype="Int64"),
    "number_of_srlgs": pd.Series(dtype="Int64"),
    "better": pd.Series(dtype="Int64"),
    "safer": pd.Series(dtype="Int64"),
    "shorter": pd.Series(dtype="Int64"),
    "same": pd.Series(dtype="Int64"),
    "worse": pd.Series(dtype="Int64"),
    "avg_unav_improvement": pd.Series(dtype = float),
    "avg_bandwidth_increase": pd.Series(dtype = float),
    "best_unav_improvement": pd.Series(dtype = float),
    "best_bandwidth_increase": pd.Series(dtype=float),
    "unav_over_lb_min": pd.Series(dtype=float),
    "unav_over_lb_max": pd.Series(dtype=float),
    "unav_over_lb_avg": pd.Series(dtype=float),
    "unav_over_lb_median": pd.Series(dtype=float),
    "other_over_lb_min": pd.Series(dtype=float),
    "other_over_lb_max": pd.Series(dtype=float),
    "other_over_lb_avg": pd.Series(dtype=float),
    "other_over_lb_median": pd.Series(dtype=float),
})

# Set graph_name to index
df = df.set_index("graph_name")

# Fill it with the graph data
for graph_name in solutions.keys():
    G = solutions[graph_name]["graph"]
    df.loc[graph_name, "number_of_nodes"] = len(G.nodes())
    df.loc[graph_name, "number_of_edges"] = len(G.edges())
    df.loc[graph_name, "number_of_srlgs"] = len(G.graph['srlgs'])

# Fill emptry cells with zeros
df.fillna(0, inplace=True)

# loop through graphs
for graph_name in solutions.keys():
    #print(graph_name)
    #print(f"Graph: {graph_name}")
    G = solutions[graph_name]["graph"]
    srlgs = G.graph["srlgs"]
    for edge in G.edges():
        G.edges[edge]["weight"] = 1

    lower_bound_ratios = []
    other_bound_ratios = []
    # loop through terminals
    for terminal_index, (s, t) in enumerate(solutions[graph_name]["terminals"].keys()):
        # get baseline solution (node disjoint paths)
        try:
            disjoint_paths = suurballe_node_disjoint(G, s, t)
        except nx.NetworkXNoPath:
            # edge case for the only node that is not 2-connected
            # instead, take its neighbor
            temp_s = s
            temp_t = t
            if temp_s == 23: temp_s = 17
            if temp_t == 23: temp_t = 17
            disjoint_paths = suurballe_node_disjoint(G, temp_s, temp_t)
            for path in disjoint_paths:
                if path[0] == 17:
                    path.insert(0, 23)
                elif path[-1] == 17:
                    path.append(23)
        #print(f"{terminal_index}\t{s}\t{t}")

        # get the failure probability magnitudes and lengths of the disjoint paths
        base_prob = 1-get_surviving_probabilities_of_walks(G, disjoint_paths)[-2]
        base_bandwidth = round(sum(len(disjoint_path) - 1 for disjoint_path in disjoint_paths), 4)

        # get the safest paths
        shortest_path = nx.shortest_path(G, s, t)
        shortest_prob = 1-get_surviving_probabilities_of_walks(G, [shortest_path])[-1]
        shortest_bandwidth = round(len(shortest_path) - 1, 4)

        # now we should determine the number of solutions for different cases
        our_probs = []
        our_bandwidths = []
        for n in range(2, 11):
            walks = solutions[graph_name]["terminals"][(s, t)]["min_cutting_prob"][(n, n-1)]["walks"]
            prob = 1-get_surviving_probabilities_of_walks(G, walks)[-2]
            bandwidth = round(sum(len(walk) - 1 for walk in walks) / (n-1), 4)
            if prob > shortest_prob:
                break
            our_probs.append(prob)
            our_bandwidths.append(bandwidth)
            if n == 2 and graph_name == graph_to_check and (s,t) in terminals_to_check:
                print(f"({s}, {t}): {prob}")
        #print(f"base_prob: {base_prob}")
        min_prob = float("inf")
        min_prob_bandwidth = 0
        for prob, bandwidth in zip(our_probs, our_bandwidths):
            if prob < min_prob:
                min_prob = prob
                min_prob_bandwidth = bandwidth
        #print(f"min_prob: {min_prob}")
        prob_diff = base_prob - min_prob
        bandwidth_diff = min_prob_bandwidth - base_bandwidth
        prob_imp = (prob_diff / base_prob) * 100
        bandwidth_imp = (bandwidth_diff / base_bandwidth) * 100
        if graph_name == graph_to_check and (s,t) in terminals_to_check:
            print(prob_imp, bandwidth_imp)
        

        if prob_imp > df.loc[graph_name, "best_unav_improvement"]:
            df.loc[graph_name, "best_unav_improvement"] = prob_imp
            df.loc[graph_name, "best_bandwidth_increase"] = bandwidth_imp

        df.loc[graph_name, "avg_unav_improvement"] += prob_imp/50
        df.loc[graph_name, "avg_bandwidth_increase"] += bandwidth_imp/50

        # Lower bound statistics
        cutting_srlgs = []
        for srlg in G.graph["srlgs"]:
            # Create a copy of the graph to avoid modifying the original
            G_copy = G.copy()
            # Remove the edges in the current SRLG
            for edge_id in set(srlg["edges"]):
                u, v = G.graph["edge_by_id"][edge_id]
                G_copy.remove_edge(u, v)

            # Check if s_id and t_id are still connected
            if not nx.has_path(G_copy, s, t):
                cutting_srlgs.append(srlg)
        min_prob = sum(srlg["probability"] for srlg in cutting_srlgs)
        solution = solutions[graph_name]["terminals"][(s, t)][method][(2, 1)]
        prob = 1-get_surviving_probabilities_of_walks(G, solution["walks"])[-2]
        lower_bound_ratios.append(prob / min_prob)
        if round(base_prob-min_prob, 10) == 0:
            if round(prob-min_prob, 10) == 0:
                other_bound_ratios.append(1)
            else: 
                print("skipped")
        else:
            other_bound_ratios.append(round(prob-min_prob, 10) / round(base_prob-min_prob, 10))
            if ((round(prob-min_prob, 10) / round(base_prob-min_prob, 10)) -1 ) * 100 > 10000:
                print(s, t)

        # Make statistics of the number of different cases
        # +=2 because we have 50 inputs, and want the results in %
        same = any(prob == base_prob and bandwidth == base_bandwidth
            for prob, bandwidth in zip(our_probs, our_bandwidths)
        )
        if same: 
            df.loc[graph_name, "same"] += 2
        better = any(
            prob < base_prob and bandwidth < base_bandwidth
            for prob, bandwidth in zip(our_probs, our_bandwidths)
        )
        if better: # there is a better solution
            df.loc[graph_name, "better"] += 2
            continue 
        shorter = any(bandwidth < base_bandwidth for bandwidth in our_bandwidths)
        safer = any(prob < base_prob for prob in our_probs)
        if shorter or safer: # there is a shorter or safer solution
            if shorter:
                df.loc[graph_name, "shorter"] += 2
            if safer:
                df.loc[graph_name, "safer"] += 2
            continue
        if not same: # we have only worse solutions
            df.loc[graph_name, "worse"] += 2
            continue

    df.loc[graph_name, "unav_over_lb_min"] = (min(lower_bound_ratios) -1) * 100
    df.loc[graph_name, "unav_over_lb_max"] = (max(lower_bound_ratios) -1 ) * 100
    df.loc[graph_name, "unav_over_lb_avg"] = (sum(lower_bound_ratios) / len(lower_bound_ratios) -1) * 100
    df.loc[graph_name, "unav_over_lb_median"] = (statistics.median(lower_bound_ratios) -1)*100
    df.loc[graph_name, "other_over_lb_min"] = (min(other_bound_ratios)-1) * 100
    df.loc[graph_name, "other_over_lb_max"] = (max(other_bound_ratios)-1) * 100
    df.loc[graph_name, "other_over_lb_avg"] = (sum(other_bound_ratios) / len(other_bound_ratios) - 1) * 100
    df.loc[graph_name, "other_over_lb_median"] = (statistics.median(other_bound_ratios) - 1 )*100

df = df.round(2)
print(df)
output_path = os.path.join("figures", "htmls", "new_data_shortest.html")
df.to_html(output_path)

19 36
(4, 64): 0.00104951403489828
0.0023400475492008402 -13.333333333333334
(4, 29): 0.0005247458359964874
0.018941445835032537 -3.8461538461538463
(19, 29): 0.0005282264499182343
0.25330017949519695 9.523809523809524
(16, 29): 0.0005248026457349564
0.1899028385753483 -4.0
                                number_of_nodes  number_of_edges  \
graph_name                                                         
22_optic_eu_latlon                           22               45   
26_usanet_latlon                             26               43   
28_optic_eu_latlon                           28               41   
37_eu_latlon                                 37               57   
39_optic_north_american_latlon               39               61   
79_optic_nfsnet_latlon                       79              108   
25_interoute_latlon                          25               34   

                                number_of_srlgs  better  safer  shorter  same  \
graph_name                     

#### Independent

In [13]:
# Read test data
path = os.path.join("results", "short_solutions_0.01.pkl")
method = "min_cutting_prob"
solutions = {}
with open(path, "rb") as f:
    solutions = pickle.load(f)

# Make pandas dataframe
df = pd.DataFrame({
    "graph_name": pd.Series(dtype = str),
    "number_of_nodes": pd.Series(dtype="Int64"),
    "number_of_edges": pd.Series(dtype="Int64"),
    "number_of_srlgs": pd.Series(dtype="Int64"),
    "better": pd.Series(dtype="Int64"),
    "safer": pd.Series(dtype="Int64"),
    "shorter": pd.Series(dtype="Int64"),
    "same": pd.Series(dtype="Int64"),
    "worse": pd.Series(dtype="Int64"),
    "avg_unav_improvement": pd.Series(dtype = float),
    "avg_bandwidth_increase": pd.Series(dtype = float),
    "best_unav_improvement": pd.Series(dtype = float),
    "best_bandwidth_increase": pd.Series(dtype=float),
    "unav_over_lb_min": pd.Series(dtype=float),
    "unav_over_lb_max": pd.Series(dtype=float),
    "unav_over_lb_avg": pd.Series(dtype=float),
    "unav_over_lb_median": pd.Series(dtype=float),
})

# Set graph_name to index
df = df.set_index("graph_name")

# Fill it with the graph data
for graph_name in solutions.keys():
    G = solutions[graph_name]["graph"]
    df.loc[graph_name, "number_of_nodes"] = len(G.nodes())
    df.loc[graph_name, "number_of_edges"] = len(G.edges())
    df.loc[graph_name, "number_of_srlgs"] = len(G.graph['srlgs'])

# Fill emptry cells with zeros
df.fillna(0, inplace=True)

# loop through graphs
for graph_name in solutions.keys():
    #print(graph_name)
    #print(f"Graph: {graph_name}")
    G = solutions[graph_name]["graph"]
    srlgs = G.graph["srlgs"]
    for edge in G.edges():
        intersecting_srlgs = [
            srlg for srlg in srlgs if G.edges[edge]["id"] in srlg["edges"]
        ]
        G.edges[edge]["weight"] = -np.log10(
            1 - sum(srlg["probability"] for srlg in intersecting_srlgs)
        )

    lower_bound_ratios = []
    # loop through terminals
    for terminal_index, (s, t) in enumerate(solutions[graph_name]["terminals"].keys()):
        # get baseline solution (node disjoint paths)
        try:
            disjoint_paths = suurballe_node_disjoint(G, s, t)
        except nx.NetworkXNoPath:
            # edge case for the only node that is not 2-connected
            # instead, take its neighbor
            temp_s = s
            temp_t = t
            if temp_s == 23: temp_s = 17
            if temp_t == 23: temp_t = 17
            disjoint_paths = suurballe_node_disjoint(G, temp_s, temp_t)
            for path in disjoint_paths:
                if path[0] == 17:
                    path.insert(0, 23)
                elif path[-1] == 17:
                    path.append(23)
        #print(f"{terminal_index}\t{s}\t{t}")

        # get the failure probability magnitudes and lengths of the disjoint paths
        base_prob = 1-get_surviving_probabilities_of_walks(G, disjoint_paths)[-2]
        base_bandwidth = round(sum(len(disjoint_path) - 1 for disjoint_path in disjoint_paths), 4)

        # get the safest paths
        shortest_path = nx.shortest_path(G, s, t)
        shortest_prob = 1-get_surviving_probabilities_of_walks(G, [shortest_path])[-1]
        shortest_bandwidth = round(len(shortest_path) - 1, 4)

        # now we should determine the number of solutions for different cases
        our_probs = []
        our_bandwidths = []
        for n in range(2, 11):
            walks = solutions[graph_name]["terminals"][(s, t)]["min_cutting_prob"][(n, n-1)]["walks"]
            prob = 1-get_surviving_probabilities_of_walks(G, walks)[-2]
            bandwidth = round(sum(len(walk) - 1 for walk in walks) / (n-1), 4)
            if prob > shortest_prob:
                break
            our_probs.append(prob)
            our_bandwidths.append(bandwidth)
        #print(f"base_prob: {base_prob}")
        min_prob = float("inf")
        min_prob_bandwidth = 0
        for prob, bandwidth in zip(our_probs, our_bandwidths):
            if prob < min_prob:
                min_prob = prob
                min_prob_bandwidth = bandwidth
        #print(f"min_prob: {min_prob}")
        prob_diff = base_prob - min_prob
        bandwidth_diff = min_prob_bandwidth - base_bandwidth
        prob_imp = (prob_diff / base_prob) * 100
        bandwidth_imp = (bandwidth_diff / base_bandwidth) * 100

        if prob_imp > df.loc[graph_name, "best_unav_improvement"]:
            df.loc[graph_name, "best_unav_improvement"] = prob_imp
            df.loc[graph_name, "best_bandwidth_increase"] = bandwidth_imp

        df.loc[graph_name, "avg_unav_improvement"] += prob_imp/50
        df.loc[graph_name, "avg_bandwidth_increase"] += bandwidth_imp/50

        # Lower bound statistics
        cutting_srlgs = []
        for srlg in G.graph["srlgs"]:
            # Create a copy of the graph to avoid modifying the original
            G_copy = G.copy()
            # Remove the edges in the current SRLG
            for edge_id in set(srlg["edges"]):
                u, v = G.graph["edge_by_id"][edge_id]
                G_copy.remove_edge(u, v)

            # Check if s_id and t_id are still connected
            if not nx.has_path(G_copy, s, t):
                cutting_srlgs.append(srlg)
        min_prob = sum(srlg["probability"] for srlg in cutting_srlgs)
        solution = solutions[graph_name]["terminals"][(s, t)][method][(2, 1)]
        prob = 1-get_surviving_probabilities_of_walks(G, solution["walks"])[-2]
        lower_bound_ratios.append(prob / min_prob)

        # Make statistics of the number of different cases
        # +=2 because we have 50 inputs, and want the results in %
        same = any(prob == base_prob and bandwidth == base_bandwidth
            for prob, bandwidth in zip(our_probs, our_bandwidths)
        )
        if same: 
            df.loc[graph_name, "same"] += 2
        better = any(
            prob < base_prob and bandwidth < base_bandwidth
            for prob, bandwidth in zip(our_probs, our_bandwidths)
        )
        if better: # there is a better solution
            df.loc[graph_name, "better"] += 2
            continue 
        shorter = any(bandwidth < base_bandwidth for bandwidth in our_bandwidths)
        safer = any(prob < base_prob for prob in our_probs)
        if shorter or safer: # there is a shorter or safer solution
            if shorter:
                df.loc[graph_name, "shorter"] += 2
            if safer:
                df.loc[graph_name, "safer"] += 2
            continue
        if not same: # we have only worse solutions
            df.loc[graph_name, "worse"] += 2
            continue

    df.loc[graph_name, "unav_over_lb_min"] = (min(lower_bound_ratios) -1) * 100
    df.loc[graph_name, "unav_over_lb_max"] = (max(lower_bound_ratios) -1 ) * 100
    df.loc[graph_name, "unav_over_lb_avg"] = (sum(lower_bound_ratios) / len(lower_bound_ratios) -1) * 100
    df.loc[graph_name, "unav_over_lb_median"] = (statistics.median(lower_bound_ratios) -1)*100

df = df.round(2)
print(df)
output_path = os.path.join("figures", "htmls", "new_data_independent.html")
df.to_html(output_path)

                                number_of_nodes  number_of_edges  \
graph_name                                                         
22_optic_eu_latlon                           22               45   
26_usanet_latlon                             26               43   
28_optic_eu_latlon                           28               41   
37_eu_latlon                                 37               57   
39_optic_north_american_latlon               39               61   
79_optic_nfsnet_latlon                       79              108   
25_interoute_latlon                          25               34   

                                number_of_srlgs  better  safer  shorter  same  \
graph_name                                                                      
22_optic_eu_latlon                          202      12     42       76    32   
26_usanet_latlon                            246       0     30       80    68   
28_optic_eu_latlon                          149       8     34 

### Compare Basic and Advanced Heuristics

In [8]:
import networkx as nx

def _build_flow_graph(G, s, t, k):
    """
    Transform G (undirected) into a directed graph for min-cost flow,
    splitting each node into in/out nodes to simulate node capacity of 1.
    """
    G_flow = nx.DiGraph()

    for node in G.nodes():
        G_flow.add_node(f"{node}_in")
        G_flow.add_node(f"{node}_out")
        G_flow.add_edge(f"{node}_in", f"{node}_out", capacity=1, weight=0)

    for u, v, data in G.edges(data=True):
        weight = data.get("weight", 1)
        G_flow.add_edge(f"{u}_out", f"{v}_in", capacity=1, weight=weight)
        G_flow.add_edge(f"{v}_out", f"{u}_in", capacity=1, weight=weight)

    # demand is at s_out (source), t_in (sink)
    G_flow.nodes[f"{s}_out"]["demand"] = -k
    G_flow.nodes[f"{t}_in"]["demand"] = k

    return G_flow

def _extract_paths_from_flow(G_flow, s, t, k, flow_dict):
    """
    Extracts the k node-disjoint paths from the flow dictionary.
    """
    paths = []
    used = set()

    for _ in range(k):
        path = []
        current = f"{s}_out"

        while current != f"{t}_in":
            path.append(current)
            for neighbor in G_flow[current]:
                if flow_dict[current].get(neighbor, 0) > 0 and (current, neighbor) not in used:
                    used.add((current, neighbor))
                    current = neighbor
                    break
            else:
                raise ValueError("Broken path during extraction")

        path.append(f"{t}_in")

        # Convert back from *_in/out to original node names, filter duplicates
        original_path = []
        seen = set()
        for node in path:
            base = node.replace("_in", "").replace("_out", "")
            if base not in seen:
                original_path.append(int(base) if base.isdigit() else base)
                seen.add(base)

        paths.append(original_path)

    return paths

def find_k_node_disjoint_paths(G, s, t, k):
    """
    Find k node-disjoint paths from s to t in an undirected graph G,
    using min-cost flow and node-splitting.

    Parameters:
        G : undirected NetworkX graph with edge attribute "weight"
        s, t : source and target nodes
        k : number of disjoint paths to find

    Returns:
        List of k paths (each path is a list of node names)
    """
    G_flow = _build_flow_graph(G, s, t, k)

    try:
        flow_dict = nx.min_cost_flow(G_flow)
        paths = _extract_paths_from_flow(G_flow, s, t, k, flow_dict)
        return paths
    except nx.NetworkXUnfeasible:
        print(f"No feasible flow: can't find {k} node-disjoint paths from {s} to {t}.")
        return []



def make_bandwidts_failure_magnitude_figure_2(G, s, t, solutions, title):
    fig = go.Figure()
    max_y = 0
    # our solutions
    for name, solution in solutions.items():
        x = []
        y = []
        text = []

        for n in range(2, 11):
            x.append(np.log10(1-solution[(n, n-1)]['surviving_probability']))
            total_length = sum(len(walk) - 1 for walk in solution[(n,n-1)]['walks'])
            bandwidth = total_length / (n - 1)
            y.append(bandwidth)
            text.append(f"{n}")

        fig.add_trace(go.Scatter(
            x = deepcopy(x),
            y = deepcopy(y),
            mode='lines+markers+text',
            name=f'{name}',
            text = deepcopy(text),
            textposition='top center',
        ))
        max_y = max(max_y, max(y))

    max_node_disjoint_paths = len(list(nx.node_disjoint_paths(G, s, t)))

    # shortest paths
    for edge in G.edges(data=True):
        edge[2]['capacity'] = 1
        edge[2]['weight'] = 1
    x = []
    y = []
    text = []
   
    paths = find_k_node_disjoint_paths(G, s, t, 1)
    bandwidth = sum(len(path) - 1 for path in paths)
    failure_probability_magnitude = np.log10(1-get_surviving_probabilities_of_walks(G, paths)[-1])
    x.append(failure_probability_magnitude)
    y.append(bandwidth)
    text.append(f"{1}")

    for number_of_node_disjoint_paths in range(2, max_node_disjoint_paths+1):
        paths = find_k_node_disjoint_paths(G, s, t, number_of_node_disjoint_paths)
        bandwidth = sum(len(path) - 1 for path in paths) / (len(paths) - 1)
        failure_probability_magnitude = np.log10(1-get_surviving_probabilities_of_walks(G, paths)[-2])
        x.append(failure_probability_magnitude)
        y.append(bandwidth)
        text.append(f"{number_of_node_disjoint_paths}")

    fig.add_trace(go.Scatter(
        x=deepcopy(x),
        y=deepcopy(y),
        mode='markers+lines+text',
        marker = dict(symbol='diamond', size=8),
        text=deepcopy(text),
        textposition='top center',
        name=f'shortest paths',
    ))
    max_y = max(max_y, max(y))

    # safest paths
    min_cost = float('inf')
    srlgs = G.graph["srlgs"]
    for edge in G.edges():
        intersecting_srlgs = [
            srlg for srlg in srlgs if G.edges[edge]["id"] in srlg["edges"]
        ]
        G.edges[edge]["capacity"] = 1
        G.edges[edge]["weight"] = int(-np.log10(
            1 - sum(srlg["probability"] for srlg in intersecting_srlgs)
        )*10**10)
        min_cost = min(min_cost, G.edges[edge]["weight"])

    x = []
    y = []
    text = []

    paths = find_k_node_disjoint_paths(G, s, t, 1)
    bandwidth = sum(len(path) - 1 for path in paths)
    failure_probability_magnitude = np.log10(1-get_surviving_probabilities_of_walks(G, paths)[-1])
    x.append(failure_probability_magnitude)
    y.append(bandwidth)
    text.append(f"{1}")

    for number_of_node_disjoint_paths in range(2, max_node_disjoint_paths+1):
        paths = find_k_node_disjoint_paths(G, s, t, number_of_node_disjoint_paths)
        bandwidth = sum(len(path) - 1 for path in paths) / (len(paths) - 1)
        failure_probability_magnitude = np.log10(1-get_surviving_probabilities_of_walks(G, paths)[-2])
        x.append(failure_probability_magnitude)
        y.append(bandwidth)
        text.append(f"{number_of_node_disjoint_paths}")

    fig.add_trace(go.Scatter(
        x=deepcopy(x),
        y=deepcopy(y),
        mode='markers+lines+text',
        marker = dict(symbol='diamond', size=8),
        text=deepcopy(text),
        textposition='top center',
        name=f'safest paths',
    ))
    max_y = max(max_y, max(y))

    # prob lower bound
    cutting_srlgs = []
    for srlg in G.graph["srlgs"]:
        # Create a copy of the graph to avoid modifying the original
        G_copy = G.copy()
        # Remove the edges in the current SRLG
        for edge_id in srlg["edges"]:
            u, v = G.graph["edge_by_id"][edge_id]
            G_copy.remove_edge(u, v)
        # Check if s_id and t_id are still connected
        if not nx.has_path(G_copy, s, t):
            cutting_srlgs.append(srlg)

    min_prob = sum(srlg["probability"] for srlg in cutting_srlgs)
    fig.add_shape(
        type="line",
        x0=np.log10(min_prob),
        y0=0,
        x1=np.log10(min_prob),
        y1=max_y,
        line=dict(color="black", width=2, dash="dash"),
        xref="x",
        yref="y"
    )


    # update the layout of the figure
    fig.update_layout(
        title=title,
        xaxis=dict(
            title='Failure Probability Magnitude',
        ),
        yaxis = dict(
            title='Bandwidth',
           # dtick = 1.
        ),
    )

    return fig

In [9]:
with open(os.path.join("results", "improved_solutions.pkl"), "rb") as f:
    original_solutions = pickle.load(f) 

with open(os.path.join("results", "short_solutions_0.01.pkl"), "rb") as f:
    improved_solutions  = pickle.load(f) 


In [96]:
graph_name = "37_eu_latlon"
G = original_solutions[graph_name]["graph"]
method = "min_cutting_prob"
figures = []

for s, t in original_solutions[graph_name]["terminals"].keys():
    solutions = {
        "original" : original_solutions[graph_name]["terminals"][(s, t)][method],
        "improved" : improved_solutions[graph_name]["terminals"][(s, t)][method],
    }
    title = f"{method} on {graph_name} <br> with terminals ({s}, {t}), distance: {nx.shortest_path_length(G, source=s, target=t)}"
    fig = make_bandwidts_failure_magnitude_figure_2(G, s, t, solutions, title)
    figures.append(fig)

app = Dash()
# Arrange them in two columns
app.layout = html.Div([
    html.Div([
        html.Div(dcc.Graph(figure=figures[i]), style={'width': '48%', 'display': 'inline-block', 'padding': '10px'})
        for i in range(len(figures))
    ])
])
app.run(port=5054)

In [11]:
s, t = 9, 19
solutions = {
    "original" : original_solutions[graph_name]["terminals"][(s, t)][method],
    "improved" : improved_solutions[graph_name]["terminals"][(s, t)][method],
}
max_node_disjoint_paths = len(list(nx.node_disjoint_paths(G, s, t)))

for name, solution in solutions.items():
    print(name)
    for n in range(2, 11):
        prob = 1-solution[(n, n-1)]['surviving_probability']
        bandwidth = sum(len(walk) - 1 for walk in solution[(n,n-1)]['walks']) / (n - 1)
        text = n
        print(f"{prob:.6f} {bandwidth:.2f} {text}")
    print()

# shortest paths
print("shortest")
for edge in G.edges(data=True):
    edge[2]['capacity'] = 1
    edge[2]['weight'] = 1

paths = find_k_node_disjoint_paths(G, s, t, 1)
bandwidth = sum(len(path) - 1 for path in paths)
prob = 1-get_surviving_probabilities_of_walks(G, paths)[-1]
text = 1
print(f"{prob:.6f} {bandwidth:.2f} {text}")

for number_of_node_disjoint_paths in range(2, max_node_disjoint_paths+1):
    paths = find_k_node_disjoint_paths(G, s, t, number_of_node_disjoint_paths)
    bandwidth = sum(len(path) - 1 for path in paths) / (len(paths) - 1)
    prob = 1-get_surviving_probabilities_of_walks(G, paths)[-2]
    text = number_of_node_disjoint_paths
    print(f"{prob:.6f} {bandwidth:.2f} {text}")
print()

print("safest")
# safest paths
min_cost = float('inf')
srlgs = G.graph["srlgs"]
for edge in G.edges():
    intersecting_srlgs = [
        srlg for srlg in srlgs if G.edges[edge]["id"] in srlg["edges"]
    ]
    G.edges[edge]["capacity"] = 1
    G.edges[edge]["weight"] = int(-np.log10(
        1 - sum(srlg["probability"] for srlg in intersecting_srlgs)
    )*10**10)
    min_cost = min(min_cost, G.edges[edge]["weight"])
    
paths = find_k_node_disjoint_paths(G, s, t, 1)
bandwidth = sum(len(path) - 1 for path in paths)
prob = 1-get_surviving_probabilities_of_walks(G, paths)[-1]
text = 1
print(f"{prob:.6f} {bandwidth:.2f} {text}")

for number_of_node_disjoint_paths in range(2, max_node_disjoint_paths+1):
    paths = find_k_node_disjoint_paths(G, s, t, number_of_node_disjoint_paths)
    bandwidth = sum(len(path) - 1 for path in paths) / (len(paths) - 1)
    prob = 1-get_surviving_probabilities_of_walks(G, paths)[-2]
    text = number_of_node_disjoint_paths
    print(f"{prob:.6f} {bandwidth:.2f} {text}")
print()

cutting_srlgs = []
for srlg in G.graph["srlgs"]:
    # Create a copy of the graph to avoid modifying the original
    G_copy = G.copy()
    # Remove the edges in the current SRLG
    for edge_id in srlg["edges"]:
        u, v = G.graph["edge_by_id"][edge_id]
        G_copy.remove_edge(u, v)
    # Check if s_id and t_id are still connected
    if not nx.has_path(G_copy, s, t):
        cutting_srlgs.append(srlg)

min_prob = sum(srlg["probability"] for srlg in cutting_srlgs)
print(f"{min_prob:.6f}")


original
0.000513 12.00 2
0.020509 6.50 3
0.021023 6.33 4
0.046608 6.00 5
0.048663 6.40 6
0.088853 6.83 7
0.117600 7.14 8
0.156435 6.62 9
0.202660 6.33 10

improved
0.000513 6.00 2
0.020510 5.00 3
0.021023 5.33 4
0.046610 5.50 5
0.048683 5.80 6
0.088853 6.83 7
0.117070 6.86 8
0.155905 6.38 9
0.202130 6.11 10

shortest
0.056590 3.00 1
0.000513 6.00 2

safest
0.052707 3.00 1
0.000513 7.00 2

0.000513


#### With availabilities

In [66]:
def make_bandwidts_failure_magnitude_figure_3(G, s, t, solutions, title):
    """
    Figure 3:
      - x = -log10(scaled failure probability)
      - x-axis tick labels = number of nines (increasing to the right)
      - black SRLG-cut line labeled below the axis
      - per-point text = number of paths (k)
    """

    factor = G.graph['rate'] * 6 / (365 * 24)

    fig = go.Figure()
    max_y = 0
    all_x = []   # x = -log10(p_fail_scaled) values

    def xcoord_from_surv(p_surv):
        p_fail = 1 - p_surv
        p_fail_scaled = p_fail * factor
        if p_fail_scaled <= 0:
            return None
        return -np.log10(p_fail_scaled)   # "nines"

    # =========================================================
    # 1) OUR SOLUTIONS (original, improved)
    # =========================================================
    for name, solution in solutions.items():
        xs = []
        ys = []
        texts = []

        for n in range(2, 11):
            p_surv = solution[(n, n - 1)]['surviving_probability']
            xval = xcoord_from_surv(p_surv)
            if xval is None:
                continue

            xs.append(xval)
            total_len = sum(len(walk) - 1 for walk in solution[(n, n - 1)]['walks'])
            ys.append(total_len / (n - 1))
            texts.append(str(n))  # n út

            all_x.append(xval)

        if xs:
            fig.add_trace(go.Scatter(
                x=xs,
                y=ys,
                mode='lines+markers+text',
                name=name,
                text=texts,
                textposition='top center',
            ))
            max_y = max(max_y, max(ys))

    # =========================================================
    # 2) SHORTEST PATHS
    # =========================================================
    # reset edges
    for _, _, data in G.edges(data=True):
        data['capacity'] = 1
        data['weight'] = 1

    xs = []
    ys = []
    texts = []

    paths = find_k_node_disjoint_paths(G, s, t, 1)
    p_surv = get_surviving_probabilities_of_walks(G, paths)[-1]
    xv = xcoord_from_surv(p_surv)
    if xv is not None:
        xs.append(xv)
        ys.append(sum(len(p) - 1 for p in paths))
        texts.append("1")
        all_x.append(xv)

    max_ndp = len(list(nx.node_disjoint_paths(G, s, t)))

    for kpaths in range(2, max_ndp + 1):
        paths = find_k_node_disjoint_paths(G, s, t, kpaths)
        p_surv = get_surviving_probabilities_of_walks(G, paths)[-2]
        xv = xcoord_from_surv(p_surv)
        if xv is not None:
            xs.append(xv)
            ys.append(sum(len(p) - 1 for p in paths) / (len(paths) - 1))
            texts.append(str(kpaths))
            all_x.append(xv)

    if xs:
        fig.add_trace(go.Scatter(
            x=xs,
            y=ys,
            mode='lines+markers+text',
            name='shortest paths',
            marker=dict(symbol='diamond'),
            text=texts,
            textposition='top center',
        ))
        max_y = max(max_y, max(ys))

    # =========================================================
    # 3) SAFEST PATHS
    # =========================================================
    srlgs = G.graph["srlgs"]
    for edge in G.edges():
        inter = [srlg for srlg in srlgs if G.edges[edge]["id"] in srlg["edges"]]
        G.edges[edge]["capacity"] = 1
        G.edges[edge]["weight"] = int(
            -np.log10(1 - sum(s["probability"] for s in inter)) * 10**10
        )

    xs = []
    ys = []
    texts = []

    # k = 1
    paths = find_k_node_disjoint_paths(G, s, t, 1)
    p_surv = get_surviving_probabilities_of_walks(G, paths)[-1]
    xv = xcoord_from_surv(p_surv)
    if xv is not None:
        xs.append(xv)
        ys.append(sum(len(p) - 1 for p in paths))
        texts.append("1")
        all_x.append(xv)

    # k >= 2
    for kpaths in range(2, max_ndp + 1):
        paths = find_k_node_disjoint_paths(G, s, t, kpaths)
        p_surv = get_surviving_probabilities_of_walks(G, paths)[-2]
        xv = xcoord_from_surv(p_surv)
        if xv is not None:
            xs.append(xv)
            ys.append(sum(len(p) - 1 for p in paths) / (len(paths) - 1))
            texts.append(str(kpaths))
            all_x.append(xv)

    if xs:
        fig.add_trace(go.Scatter(
            x=xs,
            y=ys,
            mode='lines+markers+text',
            name='safest paths',
            marker=dict(symbol='diamond'),
            text=texts,
            textposition='top center',
        ))
        max_y = max(max_y, max(ys))

    # =========================================================
    # 4) PROB LOWER BOUND (SRLG CUT LINE)
    # =========================================================
    cutting = []
    for srlg in srlgs:
        Gc = G.copy()
        for eid in srlg["edges"]:
            u, v = G.graph["edge_by_id"][eid]
            Gc.remove_edge(u, v)
        if not nx.has_path(Gc, s, t):
            cutting.append(srlg)

    min_prob = sum(s["probability"] for s in cutting)
    p_fail_lb = min_prob * factor

    if p_fail_lb > 0:
        x_lb = -np.log10(p_fail_lb)
        fig.add_shape(
            type="line",
            x0=x_lb,
            x1=x_lb,
            y0=0,
            y1=max_y,
            line=dict(color="black", width=2, dash="dash")
        )
        fig.add_annotation(
            x=x_lb,
            y=-0.1 * max_y,
            showarrow=False,
            text=f"cut: {x_lb} nines (fail={p_fail_lb:.2e})",
            yanchor="top"
        )
        all_x.append(x_lb)

    # =========================================================
    # 5) X AXIS TICKS = whole number of nines
    # =========================================================
    if all_x:
        xmin = int(np.floor(min(all_x)))
        xmax = int(np.ceil(max(all_x)))

        tickvals = list(range(xmin, xmax + 1))
        ticktext = [f"{tv} nines" for tv in tickvals]

        fig.update_xaxes(
            tickmode="array",
            tickvals=tickvals,
            ticktext=ticktext,
            title="Number of nines (−log10 failure probability)",
        )

    fig.update_layout(
        title=title,
        yaxis=dict(title='Bandwidth'),
    )

    return fig


In [42]:
graph_name = "22_optic_eu_latlon"
G = original_solutions[graph_name]["graph"]
method = "min_cutting_prob"
figures = []

for s, t in original_solutions[graph_name]["terminals"].keys():
    solutions = {
        "original" : original_solutions[graph_name]["terminals"][(s, t)][method],
        "improved" : improved_solutions[graph_name]["terminals"][(s, t)][method],
    }
    title = f"{method} on {graph_name} <br> with terminals ({s}, {t}), distance: {nx.shortest_path_length(G, source=s, target=t)}"
    fig = make_bandwidts_failure_magnitude_figure_3(G, s, t, solutions, title)
    figures.append(fig)

app = Dash()
# Arrange them in two columns
app.layout = html.Div([
    html.Div([
        html.Div(dcc.Graph(figure=figures[i]), style={'width': '48%', 'display': 'inline-block', 'padding': '10px'})
        for i in range(len(figures))
    ])
])
app.run(port=5055)

In [44]:
s, t = 3, 18
factor = G.graph['rate'] * 6 / (365 * 24)

def xcoord_from_surv(p_surv):
    """Return x = -log10(scaled failure probability) or None if not defined."""
    p_fail = 1 - p_surv
    p_fail_scaled = p_fail * factor
    if p_fail_scaled <= 0:
        return None
    return -np.log10(p_fail_scaled)

solutions = {
    "original": original_solutions[graph_name]["terminals"][(s, t)][method],
    "improved": improved_solutions[graph_name]["terminals"][(s, t)][method],
}
max_node_disjoint_paths = len(list(nx.node_disjoint_paths(G, s, t)))

# our solutions (original / improved)
for name, solution in solutions.items():
    print(name)
    for n in range(2, 11):
        p_surv = solution[(n, n-1)]['surviving_probability']
        x = xcoord_from_surv(p_surv)
        if x is None:
            continue

        bandwidth = sum(len(walk) - 1 for walk in solution[(n,n-1)]['walks']) / (n - 1)
        text = n
        # x = number of nines (−log10 of scaled failure), y = bandwidth, text = #paths
        print(f"{x:.6f} {bandwidth:.2f} {text}")
    print()

# shortest paths
print("shortest")
for edge in G.edges(data=True):
    edge[2]['capacity'] = 1
    edge[2]['weight'] = 1

paths = find_k_node_disjoint_paths(G, s, t, 1)
bandwidth = sum(len(path) - 1 for path in paths)
p_surv = get_surviving_probabilities_of_walks(G, paths)[-1]
x = xcoord_from_surv(p_surv)
if x is not None:
    print(f"{x:.6f} {bandwidth:.2f} 1")

for number_of_node_disjoint_paths in range(2, max_node_disjoint_paths+1):
    paths = find_k_node_disjoint_paths(G, s, t, number_of_node_disjoint_paths)
    bandwidth = sum(len(path) - 1 for path in paths) / (len(paths) - 1)
    p_surv = get_surviving_probabilities_of_walks(G, paths)[-2]
    x = xcoord_from_surv(p_surv)
    if x is not None:
        print(f"{x:.6f} {bandwidth:.2f} {number_of_node_disjoint_paths}")
print()

# safest paths
print("safest")
min_cost = float('inf')
srlgs = G.graph["srlgs"]
for edge in G.edges():
    intersecting_srlgs = [
        srlg for srlg in srlgs if G.edges[edge]["id"] in srlg["edges"]
    ]
    G.edges[edge]["capacity"] = 1
    G.edges[edge]["weight"] = int(-np.log10(
        1 - sum(srlg["probability"] for srlg in intersecting_srlgs)
    ) * 10**10)
    min_cost = min(min_cost, G.edges[edge]["weight"])

paths = find_k_node_disjoint_paths(G, s, t, 1)
bandwidth = sum(len(path) - 1 for path in paths)
p_surv = get_surviving_probabilities_of_walks(G, paths)[-1]
x = xcoord_from_surv(p_surv)
if x is not None:
    print(f"{x:.6f} {bandwidth:.2f} 1")

for number_of_node_disjoint_paths in range(2, max_node_disjoint_paths+1):
    paths = find_k_node_disjoint_paths(G, s, t, number_of_node_disjoint_paths)
    bandwidth = sum(len(path) - 1 for path in paths) / (len(paths) - 1)
    p_surv = get_surviving_probabilities_of_walks(G, paths)[-2]
    x = xcoord_from_surv(p_surv)
    if x is not None:
        print(f"{x:.6f} {bandwidth:.2f} {number_of_node_disjoint_paths}")
print()

# cutting SRLGs – black vertical line in the figure
cutting_srlgs = []
for srlg in G.graph["srlgs"]:
    G_copy = G.copy()
    for edge_id in srlg["edges"]:
        u, v = G.graph["edge_by_id"][edge_id]
        G_copy.remove_edge(u, v)
    if not nx.has_path(G_copy, s, t):
        cutting_srlgs.append(srlg)

min_prob = sum(srlg["probability"] for srlg in cutting_srlgs)
p_fail_lb_scaled = min_prob * factor
if p_fail_lb_scaled > 0:
    x_cut = -np.log10(p_fail_lb_scaled)
    print(f"cut_line: min_prob={min_prob:.6f}, scaled_fail={p_fail_lb_scaled:.6e}, x={x_cut:.6f}")
else:
    print(f"cut_line: min_prob={min_prob:.6f}, scaled_fail<=0")


original
4.310445 15.00 2
4.096324 9.00 3
3.350084 7.33 4
2.930595 8.00 5
2.600832 7.40 6
2.400484 7.50 7
2.305238 7.57 8
2.221980 7.50 9
2.048810 7.22 10

improved
4.310443 8.00 2
4.093764 6.50 3
3.350079 6.67 4
2.922559 7.00 5
2.646551 6.60 6
2.411006 6.00 7
2.329889 6.86 8
2.240494 7.38 9
2.048810 7.22 10

shortest
2.633040 3.00 1
4.310413 8.00 2

safest
2.633040 3.00 1
4.310413 9.00 2

cut_line: min_prob=0.001095, scaled_fail=4.892244e-05, x=4.310492


### Trimming

In [45]:
path = os.path.join("results", "short_solutions_0.01.pkl")
with open(path, "rb") as f:
    test_short_001 = pickle.load(f)

with open(os.path.join("results", "trimmed_short_solutions_0.01.pkl"), "rb") as f:
    trimmed_test_short_001 = pickle.load(f) 

In [47]:
graph_name = "22_optic_eu_latlon"
G = trimmed_test_short_001[graph_name]["graph"]
method = "min_cutting_prob"
figures = []
for s, t in list(trimmed_test_short_001[graph_name]["terminals"].keys()):
    distance = nx.shortest_path_length(G, source=s, target=t)
    solutions = {"long": test_short_001[graph_name]["terminals"][(s, t)][method]}
    for d in range(1, 3):
        solution = trimmed_test_short_001[graph_name]["terminals"][(s, t)].get(d, None)
        if  solution is None or solution == {}:
            continue
        else:
            solutions[d] = trimmed_test_short_001[graph_name]["terminals"][(s, t)][d][method]

    title = f"{method} on {graph_name} <br> with terminals ({s}, {t}), distance: {nx.shortest_path_length(G, source=s, target=t)}"
    fig = make_bandwidts_failure_magnitude_figure_2(G, s, t, solutions, title)
    figures.append(fig)

# Save the figures to an HTML file
save_figures_to_html(figures, "22_trimming_distance_comparison_0.01.html", title="22_optic_eu_latlon bandwidths with different distances")

# plot the figures
app = Dash()
# Arrange them in two columns
app.layout = html.Div([
    html.Div([
        html.Div(dcc.Graph(figure=figures[i]), style={'width': '48%', 'display': 'inline-block', 'padding': '10px'})
        for i in range(len(figures))
    ])
])
app.run()

In [47]:
graph_name = "79_optic_nfsnet_latlon"
G = trimmed_test_short_001[graph_name]["graph"]
method = "min_cutting_prob"
figures = []
for s, t in list(trimmed_test_short_001[graph_name]["terminals"].keys()):
    distance = nx.shortest_path_length(G, source=s, target=t)
    solutions = {"long": test_short_001[graph_name]["terminals"][(s, t)][method]}
    for d in range(3,5):
        solution = trimmed_test_short_001[graph_name]["terminals"][(s, t)].get(d, None)
        if  solution is None or solution == {}:
            continue
        else:
            solutions[d] = trimmed_test_short_001[graph_name]["terminals"][(s, t)][d][method]

    title = f"{method} on {graph_name} <br> with terminals ({s}, {t}), distance: {nx.shortest_path_length(G, source=s, target=t)}"
    fig = make_bandwidts_failure_magnitude_figure_2(G, s, t, solutions, title)
    figures.append(fig)

# Save the figures to an HTML file
save_figures_to_html(figures, "79_trimming_distance_comparison_0.01.html", title="79_optic_nfsnet_latlon bandwidths with different distances")

# plot the figures
app = Dash()
# Arrange them in two columns
app.layout = html.Div([
    html.Div([
        html.Div(dcc.Graph(figure=figures[i]), style={'width': '48%', 'display': 'inline-block', 'padding': '10px'})
        for i in range(len(figures))
    ])
])
app.run()

#### With vailabilities

In [48]:
path = os.path.join("results", "short_solutions_0.01.pkl")
with open(path, "rb") as f:
    test_short_001 = pickle.load(f)

with open(os.path.join("results", "trimmed_short_solutions_0.01.pkl"), "rb") as f:
    trimmed_test_short_001 = pickle.load(f) 

In [49]:
graph_name = "22_optic_eu_latlon"
G = trimmed_test_short_001[graph_name]["graph"]
method = "min_cutting_prob"
figures = []
for s, t in list(trimmed_test_short_001[graph_name]["terminals"].keys()):
    distance = nx.shortest_path_length(G, source=s, target=t)
    solutions = {"long": test_short_001[graph_name]["terminals"][(s, t)][method]}
    for d in range(1, 3):
        solution = trimmed_test_short_001[graph_name]["terminals"][(s, t)].get(d, None)
        if  solution is None or solution == {}:
            continue
        else:
            solutions[d] = trimmed_test_short_001[graph_name]["terminals"][(s, t)][d][method]

    title = f"{method} on {graph_name} <br> with terminals ({s}, {t}), distance: {nx.shortest_path_length(G, source=s, target=t)}"
    fig = make_bandwidts_failure_magnitude_figure_3(G, s, t, solutions, title)
    figures.append(fig)

# Save the figures to an HTML file
save_figures_to_html(figures, "22_trimming_distance_comparison_0.01.html", title="22_optic_eu_latlon bandwidths with different distances")

# plot the figures
app = Dash()
# Arrange them in two columns
app.layout = html.Div([
    html.Div([
        html.Div(dcc.Graph(figure=figures[i]), style={'width': '48%', 'display': 'inline-block', 'padding': '10px'})
        for i in range(len(figures))
    ])
])
app.run()

In [ ]:
graph_name = "79_optic_nfsnet_latlon"
G = trimmed_test_short_001[graph_name]["graph"]
method = "min_cutting_prob"
figures = []
for s, t in list(trimmed_test_short_001[graph_name]["terminals"].keys()):
    distance = nx.shortest_path_length(G, source=s, target=t)
    solutionss = {"long": test_short_001[graph_name]["terminals"][(s, t)][method]}
    for d in range(3, 5):
        solution = trimmed_test_short_001[graph_name]["terminals"][(s, t)].get(d, None)
        if  solution is None or solution == {}:
            continue
        else:
            solutionss[d] = trimmed_test_short_001[graph_name]["terminals"][(s, t)][d][method]

    title = f"{method} on {graph_name} <br> with terminals ({s}, {t}), distance: {nx.shortest_path_length(G, source=s, target=t)}"
    fig = make_bandwidts_failure_magnitude_figure_3(G, s, t, solutionss, title)
    figures.append(fig)

# Save the figures to an HTML file
save_figures_to_html(figures, "22_trimming_distance_comparison_0.01.html", title="79_optic_nfsnet_latlon bandwidths with different distances")

# plot the figures
app = Dash()
# Arrange them in two columns
app.layout = html.Div([
    html.Div([
        html.Div(dcc.Graph(figure=figures[i]), style={'width': '48%', 'display': 'inline-block', 'padding': '10px'})
        for i in range(len(figures))
    ])
])
app.run()

In [73]:
s, t = 21, 49

graph_name = "79_optic_nfsnet_latlon"
G = trimmed_test_short_001[graph_name]["graph"]
method = "min_cutting_prob"
figures = []
distance = nx.shortest_path_length(G, source=s, target=t)
solutionss = {"long": test_short_001[graph_name]["terminals"][(s, t)][method]}
for d in range(3, 5):
    solution = trimmed_test_short_001[graph_name]["terminals"][(s, t)].get(d, None)
    if  solution is None or solution == {}:
        continue
    else:
        solutionss[d] = trimmed_test_short_001[graph_name]["terminals"][(s, t)][d][method]

title = f"{method} on {graph_name} <br> with terminals ({s}, {t}), distance: {nx.shortest_path_length(G, source=s, target=t)}"
fig = make_bandwidts_failure_magnitude_figure_3(G, s, t, solutionss, title)
figures.append(fig)

# Save the figures to an HTML file
save_figures_to_html(figures, "22_trimming_distance_comparison_0.01.html", title="79_optic_nfsnet_latlon bandwidths with different distances")

# plot the figures
app = Dash()
# Arrange them in two columns
app.layout = html.Div([
    html.Div([
        html.Div(dcc.Graph(figure=figures[i]), style={'width': '48%', 'display': 'inline-block', 'padding': '10px'})
        for i in range(len(figures))
    ])
])
app.run()

In [75]:
s, t = 21, 49
factor = G.graph['rate'] * 6 / (365 * 24)

def xcoord_from_surv(p_surv):
    """Return x = -log10(scaled failure probability) or None if not defined."""
    p_fail = 1 - p_surv
    p_fail_scaled = p_fail * factor
    if p_fail_scaled <= 0:
        return None
    return -np.log10(p_fail_scaled)

solutions = {
    "long": solutionss["long"],
    "4": solutionss[4],
}
max_node_disjoint_paths = len(list(nx.node_disjoint_paths(G, s, t)))

# our solutions (original / improved)
for name, solution in solutions.items():
    print(name)
    for n in range(2, 11):
        p_surv = solution[(n, n-1)]['surviving_probability']
        x = xcoord_from_surv(p_surv)
        if x is None:
            continue

        bandwidth = sum(len(walk) - 1 for walk in solution[(n,n-1)]['walks']) / (n - 1)
        text = n
        # x = number of nines (−log10 of scaled failure), y = bandwidth, text = #paths
        print(f"{x:.6f} {bandwidth:.2f} {text}")
    print()

# shortest paths
print("shortest")
for edge in G.edges(data=True):
    edge[2]['capacity'] = 1
    edge[2]['weight'] = 1

paths = find_k_node_disjoint_paths(G, s, t, 1)
bandwidth = sum(len(path) - 1 for path in paths)
p_surv = get_surviving_probabilities_of_walks(G, paths)[-1]
x = xcoord_from_surv(p_surv)
if x is not None:
    print(f"{x:.6f} {bandwidth:.2f} 1")

for number_of_node_disjoint_paths in range(2, max_node_disjoint_paths+1):
    paths = find_k_node_disjoint_paths(G, s, t, number_of_node_disjoint_paths)
    bandwidth = sum(len(path) - 1 for path in paths) / (len(paths) - 1)
    p_surv = get_surviving_probabilities_of_walks(G, paths)[-2]
    x = xcoord_from_surv(p_surv)
    if x is not None:
        print(f"{x:.6f} {bandwidth:.2f} {number_of_node_disjoint_paths}")
print()

# safest paths
print("safest")
min_cost = float('inf')
srlgs = G.graph["srlgs"]
for edge in G.edges():
    intersecting_srlgs = [
        srlg for srlg in srlgs if G.edges[edge]["id"] in srlg["edges"]
    ]
    G.edges[edge]["capacity"] = 1
    G.edges[edge]["weight"] = int(-np.log10(
        1 - sum(srlg["probability"] for srlg in intersecting_srlgs)
    ) * 10**10)
    min_cost = min(min_cost, G.edges[edge]["weight"])

paths = find_k_node_disjoint_paths(G, s, t, 1)
bandwidth = sum(len(path) - 1 for path in paths)
p_surv = get_surviving_probabilities_of_walks(G, paths)[-1]
x = xcoord_from_surv(p_surv)
if x is not None:
    print(f"{x:.6f} {bandwidth:.2f} 1")

for number_of_node_disjoint_paths in range(2, max_node_disjoint_paths+1):
    paths = find_k_node_disjoint_paths(G, s, t, number_of_node_disjoint_paths)
    bandwidth = sum(len(path) - 1 for path in paths) / (len(paths) - 1)
    p_surv = get_surviving_probabilities_of_walks(G, paths)[-2]
    x = xcoord_from_surv(p_surv)
    if x is not None:
        print(f"{x:.6f} {bandwidth:.2f} {number_of_node_disjoint_paths}")
print()

# cutting SRLGs – black vertical line in the figure
cutting_srlgs = []
for srlg in G.graph["srlgs"]:
    G_copy = G.copy()
    for edge_id in srlg["edges"]:
        u, v = G.graph["edge_by_id"][edge_id]
        G_copy.remove_edge(u, v)
    if not nx.has_path(G_copy, s, t):
        cutting_srlgs.append(srlg)

min_prob = sum(srlg["probability"] for srlg in cutting_srlgs)
p_fail_lb_scaled = min_prob * factor
if p_fail_lb_scaled > 0:
    x_cut = -np.log10(p_fail_lb_scaled)
    print(f"cut_line: min_prob={min_prob:.6f}, scaled_fail={p_fail_lb_scaled:.6e}, x={x_cut:.6f}")
else:
    print(f"cut_line: min_prob={min_prob:.6f}, scaled_fail<=0")


long
5.010952 34.00 2
4.180344 19.00 3
3.605981 21.67 4
3.260147 20.25 5
3.068633 18.60 6
2.815226 19.00 7
2.651047 18.29 8
2.529643 18.12 9
2.356302 20.00 10

4
5.010954 24.00 2
3.540576 17.00 3
3.329719 17.00 4
2.963647 16.00 5
2.876364 14.80 6
2.807318 14.50 7
2.720945 14.14 8
2.622144 14.25 9
2.550366 14.11 10

shortest
3.144157 9.00 1
5.010528 23.00 2

safest
3.144157 9.00 1
5.010954 25.00 2

cut_line: min_prob=0.001056, scaled_fail=9.750930e-06, x=5.010954
